In [ ]:
# !pip install -U transformers

In [ ]:
import torch
import numpy as np
import pickle
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import re
import json
import warnings
import csv
warnings.filterwarnings("ignore")

# 设备配置（自动适配GPU/CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
model_path = "/kaggle/input/llama3-8b-instruct/transformers/default/1/Meta-Llama-3-8B-Instruct"

# 加载Tokenizer和Model
tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    device_map="auto",
    output_hidden_states=True
)
model.eval()
print("Model and Tokenizer loaded successfully!")

In [ ]:
XP = "If you enter a building from the east side, walking west, and then take two rights and a left down corridors, which direction are you facing?"

def save_hidden_states(data_dict, filename="hidden_states_data.pkl"):
    """保存数据"""
    with open(filename, 'wb') as f:
        pickle.dump(data_dict, f)
    print(f"✓ 数据已保存至 {filename}")

def build_llama3_prompt(system_msg, user_msg):
    """构建符合Llama 3格式的prompt"""
    prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_msg}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_msg}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    return prompt

print("✅ Cell 1完成：初始化设置")

# =====================================================
# Cell 2: 修复的步骤级hidden states提取
# =====================================================

def generate_step_by_step_reasoning():
    """让模型进行分步推理，记录每个步骤的hidden states"""

    step_by_step_prompt = build_llama3_prompt(
        #"You are a helpful AI assistant that solves reasoning problems step by step. You MUST output your reasoning in the exact format: Step 1: [reasoning], Step 2: [reasoning], etc. End with 'Final Answer: [direction]'",
        "You are a helpful AI assistant that solves reasoning problems step by step.",
        f"Problem: {XP}\n\nPlease solve this step by step and and provide the final answer:"
    )
    
    print("生成分步推理过程...")
    inputs = tokenizer(step_by_step_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            output_hidden_states=True,
            return_dict_in_generate=True
        )
    
    # 获取完整响应
    full_response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    assistant_response = full_response.split("assistant")[-1].strip() if "assistant" in full_response else full_response
    
    print(f"模型分步推理响应:\n{assistant_response}")
    
    # 提取步骤级hidden states
    step_data = extract_step_level_hidden_states(outputs, inputs.input_ids.shape[1], assistant_response)
    
    return assistant_response, step_data

def extract_step_level_hidden_states(outputs, prompt_length, generated_text):
    """精确提取每个步骤的hidden states"""
    
    # 解析步骤结构
    steps = parse_step_structure(generated_text)
    
    step_data = []
    generated_ids = outputs.sequences[0, prompt_length:]  # 只取生成的token（排除prompt）
    full_generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    
    print(f"完整生成文本长度: {len(full_generated_text)} 字符")
    print(f"生成token数量: {len(generated_ids)}")
    # 【关键修复1】hidden states长度 = 生成的token数量（generated_ids的长度）
    total_hidden_length = len(generated_ids)  
    print(f"生成token总长度（hidden states参考）: {total_hidden_length}")
    
    # 计算每个步骤的精确token位置
    current_position = 0
    for step_num, step_text in steps.items():
        step_text_clean = step_text.strip()
        step_start = full_generated_text.find(step_text_clean, current_position)
        
        if step_start == -1:
            full_text_clean = " ".join(full_generated_text.split())
            step_text_clean_no_space = " ".join(step_text_clean.split())
            step_start = full_text_clean.find(step_text_clean_no_space, current_position)
            if step_start == -1:
                print(f"❌ 无法找到步骤 {step_num}: {step_text[:50]}...")
                continue
        
        # 计算步骤文本的token长度
        step_tokens = tokenizer.encode(step_text_clean, add_special_tokens=False)
        step_token_length = len(step_tokens)
        
        if step_token_length == 0:
            print(f"❌ 步骤 {step_num} 无法编码为tokens")
            continue
        
        # 【关键修复2】正确计算token位置（基于生成的文本，而非整个prompt）
        text_before_step = full_generated_text[:step_start]
        tokens_before_step = tokenizer.encode(text_before_step, add_special_tokens=False)
        step_start_token_idx = len(tokens_before_step)
        step_end_token_idx = step_start_token_idx + step_token_length - 1
        
        # 【关键修复3】彻底解决结束位置<开始位置的问题（直接舍弃异常步骤，避免强制修正）
        if step_end_token_idx < step_start_token_idx:
            print(f"❌ 步骤 {step_num} token位置计算异常（结束<开始），跳过该步骤")
            current_position = step_start + len(step_text_clean)
            continue
        
        # 检查是否超出生成的token范围（真正的超出判断）
        if step_end_token_idx >= total_hidden_length:
            print(f"⚠️ 步骤 {step_num} 超出生成token范围（{step_end_token_idx} ≥ {total_hidden_length}），使用最后一个位置")
            step_end_token_idx = total_hidden_length - 1
        
        # 获取步骤结束位置的hidden state（注意：outputs.hidden_states包含prompt的token，需偏移prompt_length）
        actual_hidden_idx = prompt_length + step_end_token_idx
        if actual_hidden_idx >= len(outputs.hidden_states[-1]):
            actual_hidden_idx = len(outputs.hidden_states[-1]) - 1
        step_hidden = outputs.hidden_states[-1][actual_hidden_idx][0, -1, :].cpu().numpy()
        
        step_data.append({
            'step_number': step_num,
            'step_text': step_text,
            'hidden_state': step_hidden,
            'token_start': step_start_token_idx,
            'token_end': step_end_token_idx,
            'token_length': step_token_length,
            'text_position': step_start
        })
        
        print(f"  提取步骤 {step_num}:")
        print(f"    文本位置: {step_start}-{step_start + len(step_text_clean)}")
        print(f"    Token位置: {step_start_token_idx}-{step_end_token_idx}")
        print(f"    Token长度: {step_token_length}")
        print(f"    文本预览: {step_text[:60]}...")
        
        current_position = step_start + len(step_text_clean)
    
    # 验证提取结果
    if step_data:
        print(f"\n✓ 成功提取 {len(step_data)} 个步骤的hidden states")
        for step in step_data:
            print(f"  步骤 {step['step_number']}: token位置={step['token_start']}-{step['token_end']}, hidden shape={step['hidden_state'].shape}")
    else:
        print("❌ 未能提取任何步骤的hidden states")
        # 备用方案：使用最后一个token的hidden state
        if len(generated_ids) > 0:
            actual_hidden_idx = prompt_length + len(generated_ids) - 1
            if actual_hidden_idx >= len(outputs.hidden_states[-1]):
                actual_hidden_idx = len(outputs.hidden_states[-1]) - 1
            last_hidden = outputs.hidden_states[-1][actual_hidden_idx][0, -1, :].cpu().numpy()
            step_data.append({
                'step_number': 'step_1',
                'step_text': generated_text,
                'hidden_state': last_hidden,
                'token_start': 0,
                'token_end': len(generated_ids) - 1,
                'token_length': len(generated_ids),
                'text_position': 0
            })
            print("✓ 使用备用方案：整个响应作为一个步骤")
    
    return step_data

def parse_step_structure(text):
    """更鲁棒的步骤结构解析，支持多种格式"""
    steps = {}
    
    # 支持多种步骤格式
    step_patterns = [
        # 格式1: Step 1: 内容
        r'(Step\s*\d+\s*:)(.*?)(?=Step\s*\d+\s*:|Final Answer:|$)',
        # 格式2: 1. 内容  
        r'(\d+\.\s*)(.*?)(?=\d+\.\s*|Final Answer:|$)',
        # 格式3: 数字) 内容
        r'(\d+\)\s*)(.*?)(?=\d+\)\s*|Final Answer:|$)',
        # 格式4: Final Answer
        r'(Final Answer:\s*)(.*?)(?=$)'
    ]
    
    all_matches = []
    for pattern in step_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)
        for match in matches:
            # 计算匹配项在原文中的位置
            header, content = match[0], match[1]
            start_pos = text.find(header)
            if start_pos != -1:
                all_matches.append((start_pos, header.strip(), content.strip()))
    
    # 按位置排序
    all_matches.sort(key=lambda x: x[0])
    
    for i, (pos, header, content) in enumerate(all_matches):
        if not content:
            continue
            
        # 确定步骤键
        if "final" in header.lower() or "answer" in header.lower():
            step_key = "final"
        else:
            step_key = f"step_{i+1}"
        
        # 拼接完整步骤文本
        step_text = f"{header} {content}".strip()
        steps[step_key] = step_text
    
    # 如果还是没有找到结构化的步骤，使用基于逻辑的启发式分割
    if len(steps) <= 1:  # 只找到final answer
        steps = heuristic_step_splitting(text)
    
    print(f"解析出 {len(steps)} 个步骤:")
    for step_key, step_text in steps.items():
        print(f"  {step_key}: {step_text[:80]}...")
    
    return steps

def heuristic_step_splitting(text):
    """启发式步骤分割，当正则匹配失败时使用"""
    steps = {}
    
    # 基于常见推理模式分割
    lines = text.split('\n')
    step_count = 1
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        # 检测步骤开始的模式
        if (re.match(r'^\d+\.', line) or 
            re.match(r'^Step\s*\d+', line, re.I) or
            re.match(r'^\d+\)', line) or
            len(line) > 20 and any(keyword in line.lower() for keyword in ['first', 'then', 'next', 'after', 'finally'])):
            
            if 'final' not in line.lower() and 'answer' not in line.lower():
                steps[f'step_{step_count}'] = line
                step_count += 1
            else:
                steps['final'] = line
    
    # 如果还是太少，尝试按句子分割
    if len(steps) <= 1:
        sentences = re.split(r'[.!?]+', text)
        for sentence in sentences:
            sentence = sentence.strip()
            if len(sentence) > 15 and not sentence.startswith(('So', 'Therefore', 'Thus')):
                if step_count <= 4:  # 最多4个步骤
                    steps[f'step_{step_count}'] = sentence
                    step_count += 1
    
    return steps

print("✅ Cell 2完成：修复的步骤级hidden states提取")

# =====================================================
# Cell 3: 跳步识别函数
# =====================================================

def identify_skipped_steps(step_by_step_response):
    """使用LLM Judge识别具体跳过的步骤（优化误判）"""
    
    expected_complete_reasoning = [
        "Step 1: Enter from east side, initially facing east, then start walking west (new direction: west).",
        "Step 2: Take first right turn (from west → south).",
        "Step 3: Take second right turn (from south → east).", 
        "Step 4: Take left turn (from east → north).",
        "Final Answer: north"
    ]
    
    judge_prompt = build_llama3_prompt(
        "You are an expert reasoning analyzer. Analyze if any LOGICAL CONTENT is skipped (not just formatting differences). Allow steps to be merged if all core logic is included. Only mark steps as skipped if critical direction change logic is missing.",
        f"""Problem: {XP}

Expected Complete Reasoning (focus on CORE LOGIC, not exact formatting/step numbers):
{chr(10).join(expected_complete_reasoning)}

Actual Reasoning:
{step_by_step_response}

Rules for Analysis:
1. If the actual reasoning includes ALL direction change logic (even in merged steps), mark SKIPPED_STEPS as "none".
2. Only mark steps as skipped if specific direction change logic is missing.
3. Ignore differences in step numbering, wording, or formatting.

Provide your analysis in this exact format:
SKIPPED_STEPS: [step numbers separated by commas, or "none" if no skip]
SKIPPED_CONTENT: [description of missing logic, or "none" if no skip]
SKIPPING_LOCATION: [between step X and step Y, or "none" if no skip]"""
    )
    
    inputs = tokenizer(judge_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            num_return_sequences=1,
            do_sample=False,
            temperature=0.0,  # 确定性生成，避免随机误判
            return_dict_in_generate=True
        )
    
    judge_response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    judge_analysis = judge_response.split("assistant")[-1].strip() if "assistant" in judge_response else judge_response
    
    print(f"LLM Judge分析结果:\n{judge_analysis}")
    
    # 解析分析结果
    skipping_info = parse_skipping_analysis(judge_analysis)
    
    return skipping_info, judge_analysis

def parse_skipping_analysis(analysis_text):
    """解析LLM Judge的跳步分析结果"""
    skipping_info = {
        'skipped_steps': [],
        'skipped_content': '',
        'skipping_location': '',
        'from_step': None,
        'to_step': None
    }
    
    # 提取跳过的步骤编号
    steps_match = re.search(r'SKIPPED_STEPS:\s*([^\n]+)', analysis_text)
    if steps_match:
        steps_str = steps_match.group(1)
        # 提取数字
        step_numbers = re.findall(r'\d+', steps_str)
        skipping_info['skipped_steps'] = [f"step_{num}" for num in step_numbers]
    
    # 提取跳过内容
    content_match = re.search(r'SKIPPED_CONTENT:\s*([^\n]+)', analysis_text)
    if content_match:
        skipping_info['skipped_content'] = content_match.group(1).strip()
    
    # 提取跳过位置
    location_match = re.search(r'SKIPPING_LOCATION:\s*([^\n]+)', analysis_text)
    if location_match:
        location_str = location_match.group(1)
        skipping_info['skipping_location'] = location_str.strip()
        
        # 解析从哪个步骤跳到哪个步骤
        step_matches = re.findall(r'step\s*(\d+)', location_str.lower())
        if len(step_matches) >= 2:
            skipping_info['from_step'] = f"step_{step_matches[0]}"
            skipping_info['to_step'] = f"step_{step_matches[1]}"
    
    return skipping_info

print("✅ Cell 3完成：跳步识别函数定义")

# =====================================================
# Cell 4: LLM对比路径生成
# =====================================================

def generate_comparison_paths_with_llm(skipping_info, step_data, tokenizer, model):
    """
    ✅ 控制变量版：只在 LLM Judge 判断为“有跳步”时工作
      - 固定上下文（前 N-1 步）
      - 仅生成被跳过那一步的 3 风格 × 3 次
      - 与原始“跳步后”一步 hidden state 做 cosine 相似度
      - 返回 mean ± std
    """
    if not skipping_info.get('skipped_steps'):
        return {"status": "no_skip"}

    skipped_step_key = skipping_info['skipped_steps'][0]          # 例 "step_3"
    skipped_num = int(skipped_step_key.split('_')[1])
    target_step_key = f"step_{skipped_num + 1}"                  # 跳步后真实输出的那一步

    # 1. 构建前 N-1 步上下文
    context_lines = []
    for step in step_data:
        if step['step_number'] == skipped_step_key:
            break
        context_lines.append(step['step_text'])
    context_text = "\n".join(context_lines)

    # 2. 提取对比目标 hidden state
    target_hidden = None
    for step in step_data:
        if step['step_number'] == target_step_key:
            target_hidden = step['hidden_state']
            break
    if target_hidden is None:
        return {"status": "target_not_found"}

    # 3. 三种风格提示
    style_prompts = {
        "detailed": "Continue with a detailed reasoning step:",
        "shortcut": "Give only the direct result without reasoning:",
        "alternative": "Provide an alternative concise reasoning step:"
    }

    results = {"status": "ok", "similarities": {}}

    # 4. 每风格 3 次生成
    for style, prompt_suffix in style_prompts.items():
        sims = []
        for seed in [42, 43, 44]:
            user_msg = f"{context_text}\n\n{prompt_suffix}"
            full_prompt = build_llama3_prompt("You are a helpful assistant.", user_msg)

            inputs = tokenizer(full_prompt, return_tensors="pt", truncation=True).to(model.device)
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=50,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    seed=seed,
                    output_hidden_states=True,
                    return_dict_in_generate=True
                )

            # 最后生成 token 的 hidden state
            last_hidden = outputs.hidden_states[-1][-1][0, -1, :].cpu().numpy()
            sim = cosine_similarity(
                target_hidden.reshape(1, -1),
                last_hidden.reshape(1, -1)
            )[0][0]
            sims.append(float(sim))

        results["similarities"][style] = {
            "mean": float(np.mean(sims)),
            "std": float(np.std(sims)),
            "all": sims
        }

    return results

def generate_llm_reasoning_path(style, context, skipped_content):
    """使用LLM生成指定风格的完整推理路径"""
    
    if style == "detailed":
        system_msg = "You are a detailed reasoning assistant. Provide complete, logical steps for direction reasoning."
        user_msg = f"""Based on this context: "{context}"

The skipped step was: "{skipped_content}"

Please provide a COMPLETE and DETAILED reasoning path that includes all logical steps. Format as:
Step X: [reasoning]
Step Y: [reasoning]
..."""
    
    elif style == "alternative":
        system_msg = "You are an AI assistant. Provide an alternative complete reasoning path."
        user_msg = f"""Context: "{context}"
Skipped: "{skipped_content}"

Provide an ALTERNATIVE but equally valid complete reasoning path with all steps:"""
    
    else:  # step_by_step
        system_msg = "You are a methodical reasoning assistant."
        user_msg = f"""Given: "{context}"
Missing: "{skipped_content}"

Provide a thorough step-by-step reasoning path with no jumps:"""
    
    prompt = build_llama3_prompt(system_msg, user_msg)
    
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            return_dict_in_generate=True
        )
    
    response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    reasoning_path = parse_llm_reasoning_steps(response)
    
    print(f"  LLM生成{style}路径: {len(reasoning_path)}个步骤")
    return reasoning_path

def generate_llm_shortcut_path(context, skipped_content):
    """使用LLM生成纯跳步路径"""
    
    system_msg = "You are a concise AI assistant that provides direct answers without reasoning."
    user_msg = f"""Based on: "{context}"

Provide ONLY the final conclusion without any intermediate reasoning steps. Be direct and jump to the answer."""
    
    prompt = build_llama3_prompt(system_msg, user_msg)
    
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            return_dict_in_generate=True
        )
    
    response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    shortcut_path = [f"Direct jump: {response.split('assistant')[-1].strip()}"]
    
    print(f"  LLM生成纯跳步路径: {shortcut_path[0][:50]}...")
    return shortcut_path

def parse_llm_reasoning_steps(response_text):
    """解析LLM生成的推理步骤（去重，避免重复计数）"""
    
    # 提取assistant的响应
    if "assistant" in response_text:
        reasoning_text = response_text.split("assistant")[-1].strip()
    else:
        reasoning_text = response_text
    
    steps = []
    # 【关键修复1】只保留结构化的Step格式，过滤项目符号（避免重复解析）
    # 模式1: Step X: 格式（优先保留）
    step_pattern1 = r'(Step\s*\d+\s*:)(.*?)(?=Step\s*\d+\s*:|$)'
    matches1 = re.findall(pattern1, reasoning_text, re.IGNORECASE | re.DOTALL)
    
    # 模式2: 数字. 格式（次要保留，避免遗漏）
    step_pattern2 = r'(\d+)\.\s*([^\n]*(?=\n\d+\.|\nStep|$))'
    matches2 = re.findall(step_pattern2, reasoning_text, re.DOTALL)
    
    # 优先处理Step格式
    if matches1:
        for num, content in matches1:
            step_content = content.strip()
            if step_content and step_content not in [s.split(':')[-1].strip() for s in steps]:
                steps.append(f"{num.strip()} {step_content}")
    # 再处理数字.格式（去重）
    elif matches2:
        for num, content in matches2:
            step_content = content.strip()
            if step_content and step_content not in [s.split('.')[-1].strip() for s in steps]:
                steps.append(f"Step {num}: {step_content}")
    # 兜底：只保留非空、非重复的有效步骤
    else:
        lines = reasoning_text.split('\n')
        for line in lines:
            line = line.strip()
            # 过滤项目符号、空行、短行
            if line and len(line) > 15 and not line.startswith(('*', '-', '•')):
                if line not in steps:
                    steps.append(line)
    
    # 最终去重（确保无重复步骤）
    unique_steps = []
    seen_content = set()
    for step in steps:
        clean_content = " ".join(step.lower().split())
        if clean_content not in seen_content:
            seen_content.add(clean_content)
            unique_steps.append(step)
    
    return unique_steps

def generate_default_comparison_paths_with_llm():
    """当无法确定跳步位置时，用LLM生成默认对比路径"""
    
    print("使用LLM生成默认对比路径...")
    
    system_msg = "You are an expert at direction reasoning problems."
    user_msg = f"""Problem: {XP}

Please provide:
1. A detailed complete reasoning path
2. An alternative complete reasoning path  
3. A direct shortcut answer without reasoning

Format as:
DETAILED: [steps]
ALTERNATIVE: [steps]
SHORTCUT: [answer]"""
    
    prompt = build_llama3_prompt(system_msg, user_msg)
    
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            return_dict_in_generate=True
        )
    
    response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    llm_response = response.split("assistant")[-1].strip() if "assistant" in response else response
    
    # 解析LLM的响应
    paths = parse_default_llm_response(llm_response)
    
    return paths

def parse_default_llm_response(response_text):
    """解析LLM生成的默认路径响应"""
    
    paths = {
        'full_reasoning_path_a': [],
        'full_reasoning_path_b': [], 
        'pure_shortcut_path': [],
        'skipped_content': "unknown steps",
        'jumping_context': "default context"
    }
    
    # 尝试解析不同部分
    lines = response_text.split('\n')
    
    detailed_steps = []
    alternative_steps = []
    shortcut_answer = []
    
    current_section = None
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        if 'DETAILED:' in line.upper():
            current_section = 'detailed'
            line = line.split(':', 1)[-1].strip()
        elif 'ALTERNATIVE:' in line.upper():
            current_section = 'alternative'  
            line = line.split(':', 1)[-1].strip()
        elif 'SHORTCUT:' in line.upper():
            current_section = 'shortcut'
            line = line.split(':', 1)[-1].strip()
        
        if current_section == 'detailed' and line:
            detailed_steps.append(line)
        elif current_section == 'alternative' and line:
            alternative_steps.append(line)
        elif current_section == 'shortcut' and line:
            shortcut_answer.append(line)
    
    # 设置路径
    paths['full_reasoning_path_a'] = detailed_steps if detailed_steps else ["Step 1: Default detailed reasoning"]
    paths['full_reasoning_path_b'] = alternative_steps if alternative_steps else ["Step 1: Default alternative reasoning"]
    paths['pure_shortcut_path'] = shortcut_answer if shortcut_answer else ["Direct answer: north"]
    
    return paths

print("✅ Cell 4完成：LLM对比路径生成函数定义")

# =====================================================
# Cell 5: 改进的路径hidden states生成和相似度分析
# =====================================================

def generate_paths_hidden_states(comparison_paths, skipping_info):
    """为所有对比路径生成hidden states - 改进版本"""
    
    system_message = "You are a helpful AI assistant that reasons about direction problems."
    path_hidden_states = {}
    
    for path_name, path_steps in comparison_paths.items():
        if path_name == 'skipped_content' or path_name == 'jumping_context':
            continue
            
        print(f"\n生成路径: {path_name}")
        
        if isinstance(path_steps, list):
            # 为路径中的每个步骤生成hidden states
            step_hidden_states = []
            
            for step_idx, step_text in enumerate(path_steps):
                # 构建完整的上下文
                previous_steps = " ".join(path_steps[:step_idx])
                if previous_steps:
                    context = f"{XP}\n\n{previous_steps}\n\nContinue: {step_text}"
                else:
                    context = f"{XP}\n\n{step_text}"
                
                user_message = f"Please continue this reasoning:\n{context}"
                full_prompt = build_llama3_prompt(system_message, user_message)
                
                inputs = tokenizer(full_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
                
                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_new_tokens=10,  # 适当增加以获取稳定的hidden state
                        num_return_sequences=1,
                        do_sample=False,
                        temperature=0.0,
                        output_hidden_states=True,
                        return_dict_in_generate=True
                    )
                
                # 使用精确的hidden state提取
                generated_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
                assistant_response = generated_text.split("assistant")[-1].strip() if "assistant" in generated_text else generated_text
                
                # 提取最后一个token的hidden state
                last_hidden = outputs.hidden_states[-1][-1][0, -1, :].cpu().numpy()
                step_hidden_states.append(last_hidden)
                
                print(f"  步骤 {step_idx+1}: {step_text[:50]}...")
                print(f"    Hidden state形状: {last_hidden.shape}")
            
            path_hidden_states[path_name] = step_hidden_states
    
    return path_hidden_states

def calculate_similarity_analysis(skipping_step_hidden, path_hidden_states, step_data, skipping_info):
    """改进的相似度分析"""
    
    similarity_results = {}
    
    # 找到跳步位置的hidden state
    target_hidden = None
    target_step_name = skipping_info.get('to_step', 'step_3')
    
    for step in step_data:
        if step['step_number'] == target_step_name:
            target_hidden = step['hidden_state']
            break
    
    if target_hidden is None and step_data:
        # 使用最后一个步骤作为目标
        target_hidden = step_data[-1]['hidden_state']
        target_step_name = step_data[-1]['step_number']
    
    if target_hidden is None:
        print("❌ 无法找到目标hidden state")
        return similarity_results
    
    print(f"\n🔬 相似度分析 (目标步骤: {target_step_name})")
    print(f"目标hidden state形状: {target_hidden.shape}")
    
    # 计算与每个路径的相似度
    for path_name, path_hs_list in path_hidden_states.items():
        if not path_hs_list:
            continue
            
        path_similarities = []
        
        # 对路径中的每个步骤都计算相似度，取最高值
        for path_hs in path_hs_list:
            if path_hs.shape != target_hidden.shape:
                print(f"⚠️ 形状不匹配: {path_hs.shape} vs {target_hidden.shape}")
                continue
                
            similarity = cosine_similarity(
                target_hidden.reshape(1, -1),
                path_hs.reshape(1, -1)
            )[0][0]
            path_similarities.append(similarity)
        
        if path_similarities:
            similarity_results[path_name] = {
                'mean_similarity': np.mean(path_similarities),
                'max_similarity': np.max(path_similarities),
                'min_similarity': np.min(path_similarities),
                'all_similarities': path_similarities,
                'best_step_similarity': np.max(path_similarities)
            }
            
            print(f"  {path_name}:")
            print(f"    平均相似度 = {np.mean(path_similarities):.4f}")
            print(f"    最高相似度 = {np.max(path_similarities):.4f}")
            print(f"    最低相似度 = {np.min(path_similarities):.4f}")
        else:
            print(f"  {path_name}: 无法计算相似度")
    
    return similarity_results

print("✅ Cell 5完成：改进的相似度分析函数定义")

# =====================================================
# Cell 6: 主实验流程（新版）
# =====================================================
def main_experiment():
    print("=" * 70)
    print("步骤级别隐式推理分析实验 - 完整流程")
    print("=" * 70)

    # 阶段1: 生成分步推理
    print("\n📝 阶段1: 生成分步推理")
    step_by_step_response, step_data = generate_step_by_step_reasoning()

    # 阶段2: 识别跳步位置
    print("\n🔍 阶段2: 识别跳步位置")
    skipping_info, judge_analysis = identify_skipped_steps(step_by_step_response)

    # 阶段3: 控制变量对比路径（仅跳步时触发）
    print("\n🛣️  阶段3: 控制变量对比路径（仅跳步时触发）")
    comparison_results = generate_comparison_paths_with_llm(
        skipping_info, step_data, tokenizer, model
    )

    if comparison_results["status"] == "no_skip":
        print("✅ LLM Judge 判定无跳步，跳过对比阶段")
        comparison_paths = {}
        similarity_results = {}
    else:
        print("🔍 存在跳步，生成控制变量对比路径")
        comparison_paths = {}          # 本版不再需前面旧版字典
        similarity_results = comparison_results["similarities"]
        for style, stats in similarity_results.items():
            print(f"  {style}: mean={stats['mean']:.4f} ± {stats['std']:.4f}")

    # 阶段4: 生成路径 hidden states（仅旧版需要，新版留空）
    print("\n🧠 阶段4: 生成路径hidden states")
    path_hidden_states = {}          # 占位，不再使用

    # 阶段5: 相似度分析（新版直接取自 comparison_results）
    print("\n📊 阶段5: 相似度分析")
    # 已在上一步完成

    # 保存完整结果
    complete_results = {
        'XP': XP,
        'step_by_step_response': step_by_step_response,
        'step_data': step_data,
        'skipping_info': skipping_info,
        'judge_analysis': judge_analysis,
        'comparison_results': comparison_results,
        'similarity_results': similarity_results
    }
    save_hidden_states(complete_results, "complete_skipping_analysis.pkl")

    # 若存在跳步，额外保存跳步位置 hidden state（供方法三使用）
    if comparison_results["status"] == "ok":
        target_key = f"step_{int(skipping_info['skipped_steps'][0].split('_')[1]) + 1}"
        for step in step_data:
            if step['step_number'] == target_key:
                jump_step_hidden = step['hidden_state']
                save_hidden_states({
                    'hs_jump': jump_step_hidden,
                    'jump_step': target_key,
                    'XP': XP,
                    'skipping_info': skipping_info
                }, "method1_jump_hidden_state.pkl")
                print(f"✓ 跳步 hidden state 已保存，形状: {jump_step_hidden.shape}")
                break

    # 最终控制台摘要
    print("\n" + "=" * 70)
    print("🎯 最终分析结果")
    print("=" * 70)
    if comparison_results["status"] == "no_skip":
        print("✅ 无跳步，实验结束")
    else:
        print(f"跳步位置: {skipping_info.get('skipping_location', 'unknown')}")
        print(f"跳过内容: {skipping_info.get('skipped_content', 'unknown')}")
        print("\n控制变量相似度（mean ± std）：")
        for style, stats in similarity_results.items():
            print(f"  {style}: {stats['mean']:.4f} ± {stats['std']:.4f}")
        best_style = max(similarity_results.items(), key=lambda x: x[1]['mean'])
        print(f"\n💡 结论：跳步处内部推理最接近 '{best_style[0]}' 风格")
    return complete_results

print("🚀 开始执行完整实验...")
results = main_experiment()
print("\n🎉 实验完成！")

In [ ]:
import os

# 检查文件是否存在
csv_path = "/kaggle/input/testset/testset.csv"
if not os.path.exists(csv_path):
    print(f"文件不存在！请检查路径：{csv_path}")
    print("当前目录下的文件列表：", os.listdir("/kaggle/input/testset"))  # 打印文件夹内的文件
else:
    print("文件存在，可以正常加载")

### testset

In [ ]:
def save_hidden_states(data_dict, filename="hidden_states_data.pkl"):
    """保存数据"""
    with open(filename, 'wb') as f:
        pickle.dump(data_dict, f)

def build_llama3_prompt(system_msg, user_msg):
    """构建符合Llama 3格式的prompt"""
    return f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_msg}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_msg}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

# =====================================================
# 读取CSV测试集函数
# =====================================================
def load_testset(csv_path="/kaggle/input/testset/testset.csv"):
    """读取CSV格式的测试集"""
    testset = []
    with open(csv_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            expected_steps = row['expected_complete_reasoning'].split('||')
            testset.append({
                'id': row['id'],
                'problem': row['problem'],
                'expected_complete_reasoning': expected_steps
            })
    print(f"✓ 成功加载 {len(testset)} 个测试问题")
    return testset

# =====================================================
# 步骤级hidden states提取
# =====================================================
def extract_step_level_hidden_states(outputs, prompt_length, generated_text):
    steps = parse_step_structure(generated_text)
    step_data = []
    generated_ids = outputs.sequences[0, prompt_length:]
    full_generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    total_hidden_length = len(generated_ids)
    
    current_position = 0
    for step_num, step_text in steps.items():
        step_text_clean = step_text.strip()
        step_start = full_generated_text.find(step_text_clean, current_position)
        
        if step_start == -1:
            full_text_clean = " ".join(full_generated_text.split())
            step_text_clean_no_space = " ".join(step_text_clean.split())
            step_start = full_text_clean.find(step_text_clean_no_space, current_position)
            if step_start == -1:
                continue
        
        step_tokens = tokenizer.encode(step_text_clean, add_special_tokens=False)
        step_token_length = len(step_tokens)
        if step_token_length == 0:
            continue
        
        text_before_step = full_generated_text[:step_start]
        tokens_before_step = tokenizer.encode(text_before_step, add_special_tokens=False)
        step_start_token_idx = len(tokens_before_step)
        step_end_token_idx = step_start_token_idx + step_token_length - 1
        
        if step_end_token_idx < step_start_token_idx:
            current_position = step_start + len(step_text_clean)
            continue
        
        if step_end_token_idx >= total_hidden_length:
            step_end_token_idx = total_hidden_length - 1
        
        actual_hidden_idx = prompt_length + step_end_token_idx
        if actual_hidden_idx >= len(outputs.hidden_states[-1]):
            actual_hidden_idx = len(outputs.hidden_states[-1]) - 1
        step_hidden = outputs.hidden_states[-1][actual_hidden_idx][0, -1, :].cpu().numpy()
        
        step_data.append({
            'step_number': step_num,
            'step_text': step_text,
            'hidden_state': step_hidden,
            'token_start': step_start_token_idx,
            'token_end': step_end_token_idx,
            'token_length': step_token_length,
            'text_position': step_start
        })
        
        current_position = step_start + len(step_text_clean)
    
    if not step_data and len(generated_ids) > 0:
        actual_hidden_idx = prompt_length + len(generated_ids) - 1
        if actual_hidden_idx >= len(outputs.hidden_states[-1]):
            actual_hidden_idx = len(outputs.hidden_states[-1]) - 1
        last_hidden = outputs.hidden_states[-1][actual_hidden_idx][0, -1, :].cpu().numpy()
        step_data.append({
            'step_number': 'step_1',
            'step_text': generated_text,
            'hidden_state': last_hidden,
            'token_start': 0,
            'token_end': len(generated_ids) - 1,
            'token_length': len(generated_ids),
            'text_position': 0
        })
    
    return step_data

def parse_step_structure(text):
    steps = {}
    step_patterns = [
        r'(Step\s*\d+\s*:)(.*?)(?=Step\s*\d+\s*:|Final Answer:|$)',
        r'(\d+\.\s*)(.*?)(?=\d+\.\s*|Final Answer:|$)',
        r'(\d+\)\s*)(.*?)(?=\d+\)\s*|Final Answer:|$)',
        r'(Final Answer:\s*)(.*?)(?=$)'
    ]
    
    all_matches = []
    for pattern in step_patterns:
        matches = re.findall(pattern, text, re.IGNORECASE | re.DOTALL)
        for match in matches:
            header, content = match[0], match[1]
            start_pos = text.find(header)
            if start_pos != -1:
                all_matches.append((start_pos, header.strip(), content.strip()))
    
    all_matches.sort(key=lambda x: x[0])
    
    for i, (pos, header, content) in enumerate(all_matches):
        if not content:
            continue
        if "final" in header.lower() or "answer" in header.lower():
            step_key = "final"
        else:
            step_key = f"step_{i+1}"
        step_text = f"{header} {content}".strip()
        steps[step_key] = step_text
    
    if len(steps) <= 1:
        steps = heuristic_step_splitting(text)
    
    return steps

def heuristic_step_splitting(text):
    steps = {}
    lines = text.split('\n')
    step_count = 1
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
        if (re.match(r'^\d+\.', line) or 
            re.match(r'^Step\s*\d+', line, re.I) or
            re.match(r'^\d+\)', line) or
            len(line) > 20 and any(keyword in line.lower() for keyword in ['first', 'then', 'next', 'after', 'finally'])):
            if 'final' not in line.lower() and 'answer' not in line.lower():
                steps[f'step_{step_count}'] = line
                step_count += 1
            else:
                steps['final'] = line
    
    if len(steps) <= 1:
        sentences = re.split(r'[.!?]+', text)
        for sentence in sentences:
            sentence = sentence.strip()
            if len(sentence) > 15 and not sentence.startswith(('So', 'Therefore', 'Thus')):
                if step_count <= 4:
                    steps[f'step_{step_count}'] = sentence
                    step_count += 1
    
    return steps

# =====================================================
# 跳步识别函数
# =====================================================
def identify_skipped_steps(step_by_step_response, problem, expected_complete_reasoning):
    judge_prompt = build_llama3_prompt(
        "You are a strict reasoning analyzer. ONLY focus on comparing actual reasoning with expected core logic step by step. Missing any expected core logic = skipped step. No lenient judgment.",
        f"""Problem: {problem}

Expected Complete Reasoning (EACH STEP'S CORE LOGIC IS MANDATORY):
{chr(10).join([f"Expected Step {i+1}: {step.strip()}" for i, step in enumerate(expected_complete_reasoning)])}

Actual Reasoning:
{step_by_step_response}

Strict Rules:
1. Compare actual reasoning with EVERY expected step's core logic (ignore wording/format differences ONLY if core logic is fully included).
2. If ANY expected core logic is not found in actual reasoning → mark as skipped step.
3. For incomplete problems (actual reasoning asks for more info), mark SKIPPED_STEPS as "incomplete_problem", not "none".
4. Do NOT use "none" if actual reasoning is incomplete or missing expected logic.

Must output in this EXACT format (no extra words):
SKIPPED_STEPS: [step numbers separated by commas / "incomplete_problem" / "none" only if 100% complete]
SKIPPED_CONTENT: [missing core logic / "incomplete_problem" / "none"]
SKIPPING_LOCATION: [between step X and step Y / "incomplete_problem" / "none"]"""
    )
    
    inputs = tokenizer(judge_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            num_return_sequences=1,
            do_sample=False,
            return_dict_in_generate=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    judge_response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    judge_analysis = judge_response.split("assistant")[-1].strip() if "assistant" in judge_response else judge_response
    skipping_info = parse_skipping_analysis(judge_analysis)
    return skipping_info, judge_analysis

def parse_skipping_analysis(analysis_text):
    skipping_info = {
        'skipped_steps': [],
        'skipped_content': '',
        'skipping_location': '',
        'from_step': None,
        'to_step': None
    }
    
    steps_match = re.search(r'SKIPPED_STEPS:\s*([^\n]+)', analysis_text)
    if steps_match:
        steps_str = steps_match.group(1)
        step_numbers = re.findall(r'\d+', steps_str)
        skipping_info['skipped_steps'] = [f"step_{num}" for num in step_numbers]
    
    content_match = re.search(r'SKIPPED_CONTENT:\s*([^\n]+)', analysis_text)
    if content_match:
        skipping_info['skipped_content'] = content_match.group(1).strip()
    
    location_match = re.search(r'SKIPPING_LOCATION:\s*([^\n]+)', analysis_text)
    if location_match:
        location_str = location_match.group(1)
        skipping_info['skipping_location'] = location_str.strip()
        step_matches = re.findall(r'step\s*(\d+)', location_str.lower())
        if len(step_matches) >= 2:
            skipping_info['from_step'] = f"step_{step_matches[0]}"
            skipping_info['to_step'] = f"step_{step_matches[1]}"
    
    return skipping_info

# =====================================================
# LLM对比路径生成
# =====================================================
def generate_comparison_paths_with_llm(skipping_info, step_data, tokenizer, model):
    if not skipping_info.get('skipped_steps'):
        return {"status": "no_skip"}

    # 获取跳步前后的步骤信息（关键锚点）
    skipped_step_key = skipping_info['skipped_steps'][0]
    skipped_num = int(skipped_step_key.split('_')[1])
    prev_step_key = f"step_{skipped_num - 1}" if skipped_num > 1 else None
    target_step_key = f"step_{skipped_num + 1}"

    # 提取跳步前的步骤文本（prev_text）和跳步后的目标步骤文本（target_text）
    prev_text = ""
    for step in step_data:
        if step['step_number'] == prev_step_key:
            prev_text = step['step_text'].strip()
            break
    # 如果没有前序步骤，用问题开头作为上下文
    if not prev_text and skipped_num == 1:
        prev_text = "Initial state: Start of reasoning"

    target_text = ""
    for step in step_data:
        if step['step_number'] == target_step_key:
            target_text = step['step_text'].strip()
            break

    # 构建明确的上下文：告知模型需要补充“prev_text”到“target_text”之间的推理
    context_text = f"""You need to补充 the reasoning step between:
Previous step: {prev_text}
Next step (target): {target_text}
Please continue the reasoning logically."""

    style_prompts = {
        "detailed": "Provide a detailed reasoning step to connect the previous and next step:",
        "shortcut": "Give a direct result that bridges the previous and next step (no reasoning):",
        "alternative": "Provide an alternative concise reasoning step to connect them:"
    }

    results = {"status": "ok", "similarities": {}, "paths": {}}

    for style, prompt_suffix in style_prompts.items():
        sims = []
        path_texts = []
        for _ in range(3):
            user_msg = f"{context_text}\n\n{prompt_suffix}"
            full_prompt = build_llama3_prompt("You are a precise reasoning assistant. Only output the reasoning step, no extra words.", user_msg)

            inputs = tokenizer(full_prompt, return_tensors="pt", truncation=True).to(model.device)
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=100,  # 足够容纳单步推理
                    do_sample=True,
                    output_hidden_states=True,
                    return_dict_in_generate=True,
                    pad_token_id=tokenizer.eos_token_id
                )

            # 解码生成的路径文本（只保留推理内容）
            generated_ids = outputs.sequences[0, len(inputs.input_ids[0]):]
            path_text = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()
            # 过滤无关回复（如询问上下文的内容）
            if "help" in path_text.lower() or "context" in path_text.lower() or "provide" in path_text.lower():
                path_text = f"[Invalid: Missing reasoning to connect steps]"
            path_texts.append(path_text)

            # 计算相似度
            last_hidden = outputs.hidden_states[-1][-1][0, -1, :].cpu().numpy()
            # 确保目标步骤的hidden state存在
            target_hidden = None
            for step in step_data:
                if step['step_number'] == target_step_key:
                    target_hidden = step['hidden_state']
                    break
            if target_hidden is None:
                sims.append(0.0)
                continue
            sim = cosine_similarity(
                target_hidden.reshape(1, -1),
                last_hidden.reshape(1, -1)
            )[0][0]
            sims.append(float(sim))

        results["similarities"][style] = {
            "mean": float(np.mean(sims)),
            "std": float(np.std(sims)),
            "all": sims
        }
        results["paths"][style] = path_texts

    return results

# =====================================================
# 单个问题实验流程
# =====================================================
def main_experiment_with_details(test_sample):
    test_id = test_sample['id']
    problem = test_sample['problem']
    expected_steps = test_sample['expected_complete_reasoning']
    
    # 生成分步推理
    step_by_step_prompt = build_llama3_prompt(
        "You solve reasoning problems in a FIXED FORMAT. Follow EXACTLY: 1. Start with 'Step 1: ...' 2. Each step has only core logic 3. End with 'Final Answer: ...' 4. No extra greetings/explanations 5. Do NOT ask for more information (try your best if problem is simple)",
        f"Problem: {problem}\n\nSolve step by step in the FIXED FORMAT (only core logic, no extra words):"
    )
    inputs = tokenizer(step_by_step_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=500,
            num_return_sequences=1,
            do_sample=True,
            output_hidden_states=True,
            return_dict_in_generate=True,
            pad_token_id=tokenizer.eos_token_id
        )
    full_response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    step_by_step_response = full_response.split("assistant")[-1].strip() if "assistant" in full_response else full_response
    
    # 提取步骤级hidden states
    prompt_length = inputs.input_ids.shape[1]
    step_data = extract_step_level_hidden_states(outputs, prompt_length, step_by_step_response)
    
    # 识别跳步位置
    skipping_info, judge_analysis = identify_skipped_steps(step_by_step_response, problem, expected_steps)
    
    # 生成对比路径（包含文本）
    comparison_results = generate_comparison_paths_with_llm(skipping_info, step_data, tokenizer, model)
    
    return {
        'test_id': test_id,
        'step_by_step_response': step_by_step_response,
        'judge_analysis': judge_analysis,
        'skipping_info': skipping_info,
        'similarity_results': comparison_results.get('similarities', {}),
        'paths': comparison_results.get('paths', {})  # 新增：返回路径文本
    }

# =====================================================
# 批量运行测试集（核心修改：打印路径文本）
# =====================================================
def run_batch_test():
    testset = load_testset("/kaggle/input/testset/testset.csv")
    
    print("=" * 70)
    print(f"批量跳步分析实验 - 共 {len(testset)} 个问题")
    print("=" * 70)
    
    style_to_path = {
        "detailed": "A->B->C",
        "shortcut": "A->C",
        "alternative": "A->B1->C"
    }
    
    for test_sample in testset:
        detailed_result = main_experiment_with_details(test_sample)
        test_id = detailed_result['test_id']
        skipping_info = detailed_result['skipping_info']
        similarities = detailed_result['similarity_results']
        paths = detailed_result['paths']  # 获取路径文本
        step_by_step_response = detailed_result['step_by_step_response']
        judge_analysis = detailed_result['judge_analysis']
        
        print(f"\n【{test_id} 详细验证】")
        print(f"模型推理过程：\n{step_by_step_response[:500]}...")
        print(f"\nLLM Judge分析：\n{judge_analysis}")
        
        # 新增：如果有跳步，打印3种风格的路径文本
        if skipping_info['skipped_steps']:
            skip_step_num = skipping_info['skipped_steps'][0].split('_')[-1]
            print(f"\n【跳步{skip_step_num}的对比路径】")
            for style, path_texts in paths.items():
                print(f"\n{style_to_path[style]}（{style}风格）：")
                for i, text in enumerate(path_texts, 1):
                    print(f"  路径{i}：{text[:200]}...")  # 打印前200字符避免过长
        
        # 原有输出格式
        if skipping_info['skipped_steps']:
            if similarities:
                best_style = max(similarities.items(), key=lambda x: x[1]['mean'])[0]
                similar_path = style_to_path[best_style]
                best_similarity = round(similarities[best_style]['mean'], 4)
                print(f"\n{test_id} 跳步{skip_step_num}: 相似路径：{similar_path} 相似度：{best_similarity}")
            else:
                print(f"\n{test_id} 跳步{skip_step_num}: 相似路径：无 相似度：0.0")
        else:
            print(f"\n{test_id}: 无跳步（根据Judge分析）")
    
    print("\n" + "=" * 70)
    print("🎉 批量实验完成！")

# =====================================================
# 执行批量测试
# =====================================================
if __name__ == "__main__":
    run_batch_test()

### 方法一

### 初始化设置+工具函数

In [ ]:
XP = "If you enter a building from the east side, walking west, and then take two rights and a left down corridors, which direction are you facing?"

def save_hidden_states(data_dict, filename="hidden_states_data.pkl"):
    """保存数据"""
    with open(filename, 'wb') as f:
        pickle.dump(data_dict, f)
    print(f"✓ 数据已保存至 {filename}")

def build_llama3_prompt(system_msg, user_msg):
    """构建符合Llama 3格式的prompt"""
    prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_msg}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_msg}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    return prompt

print("✅ Cell 1完成：初始化设置")

### 生成分步推理并提取步骤级hidden states

In [ ]:
def generate_step_by_step_reasoning():
    """让模型进行分步推理，记录每个步骤的hidden states"""
    
    step_by_step_prompt = build_llama3_prompt(
        "You are a helpful AI assistant that solves direction reasoning problems step by step. You MUST output your reasoning in the exact format: Step 1: [reasoning], Step 2: [reasoning], etc. End with 'Final Answer: [direction]'",
        f"Problem: {XP}\n\nPlease solve this step by step and output each step clearly:"
    )
    
    print("生成分步推理过程...")
    inputs = tokenizer(step_by_step_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            output_hidden_states=True,
            return_dict_in_generate=True
        )
    
    # 获取完整响应
    full_response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    assistant_response = full_response.split("assistant")[-1].strip() if "assistant" in full_response else full_response
    
    print(f"模型分步推理响应:\n{assistant_response}")
    
    # 提取步骤级hidden states
    step_data = extract_step_level_hidden_states(outputs, inputs.input_ids.shape[1], assistant_response)
    
    return assistant_response, step_data

def extract_step_level_hidden_states(outputs, prompt_length, generated_text):
    """提取每个步骤的hidden states"""
    
    # 解析步骤结构
    steps = parse_step_structure(generated_text)
    
    step_data = []
    generated_ids = outputs.sequences[0, prompt_length:]
    full_generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    
    # 为每个步骤找到对应的token位置
    for step_num, step_text in steps.items():
        # 找到步骤文本在完整生成文本中的位置
        step_start = full_generated_text.find(step_text)
        if step_start == -1:
            continue
            
        # 估算token位置
        approx_token_start = step_start // 4  # 粗略估算
        approx_token_end = approx_token_start + len(step_text) // 4
        
        # 确保在范围内
        token_pos = min(approx_token_end, len(outputs.hidden_states[-1]) - 1)
        
        # 获取该位置的hidden state
        step_hidden = outputs.hidden_states[-1][token_pos][0, -1, :].cpu().numpy()
        
        step_data.append({
            'step_number': step_num,
            'step_text': step_text,
            'hidden_state': step_hidden,
            'token_position': token_pos
        })
        
        print(f"  提取步骤 {step_num}: {step_text[:50]}...")
    
    return step_data

def parse_step_structure(text):
    """解析文本中的步骤结构"""
    steps = {}
    
    # 使用正则表达式匹配 Step X: 格式
    step_pattern = r'(Step\s+\d+:|Final Answer:)([^Step]*)'
    matches = re.findall(step_pattern, text, re.IGNORECASE | re.DOTALL)
    
    for i, (step_header, step_content) in enumerate(matches):
        step_key = f"step_{i+1}" if "Final" not in step_header else "final"
        step_text = (step_header + step_content).strip()
        steps[step_key] = step_text
    
    # 如果没有找到步骤结构，整个文本作为一个步骤
    if not steps:
        steps['step_1'] = text.strip()
    
    return steps

print("✅ Cell 2完成：分步推理生成函数定义")

### LLM Judge识别跳步位置

In [ ]:
def identify_skipped_steps(step_by_step_response):
    """使用LLM Judge识别具体跳过的步骤"""
    
    expected_complete_reasoning = [
        "Step 1: Initial direction is west (entering from east walking west).",
        "Step 2: First right turn: west → north.",
        "Step 3: Second right turn: north → east.", 
        "Step 4: Left turn: east → north.",
        "Step 5: Final Answer: north"
    ]
    
    judge_prompt = build_llama3_prompt(
        "You are an expert reasoning analyzer. Analyze if any logical steps are skipped in the reasoning process. Be specific about which exact steps are missing.",
        f"""Problem: {XP}

Expected Complete Reasoning Path:
{chr(10).join(expected_complete_reasoning)}

Actual Reasoning:
{step_by_step_response}

Please analyze:
1. Which specific step numbers are skipped or missing?
2. What is the exact content of the skipped steps?
3. Between which steps does the skipping occur?

Provide your analysis in this exact format:
SKIPPED_STEPS: [step numbers]
SKIPPED_CONTENT: [description of what was skipped]
SKIPPING_LOCATION: [between step X and step Y]"""
    )
    
    inputs = tokenizer(judge_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            num_return_sequences=1,
            do_sample=False,
            temperature=0.0,
            return_dict_in_generate=True
        )
    
    judge_response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    judge_analysis = judge_response.split("assistant")[-1].strip() if "assistant" in judge_response else judge_response
    
    print(f"LLM Judge分析结果:\n{judge_analysis}")
    
    # 解析分析结果
    skipping_info = parse_skipping_analysis(judge_analysis)
    
    return skipping_info, judge_analysis

def parse_skipping_analysis(analysis_text):
    """解析LLM Judge的跳步分析结果"""
    skipping_info = {
        'skipped_steps': [],
        'skipped_content': '',
        'skipping_location': '',
        'from_step': None,
        'to_step': None
    }
    
    # 提取跳过的步骤编号
    steps_match = re.search(r'SKIPPED_STEPS:\s*([^\n]+)', analysis_text)
    if steps_match:
        steps_str = steps_match.group(1)
        # 提取数字
        step_numbers = re.findall(r'\d+', steps_str)
        skipping_info['skipped_steps'] = [f"step_{num}" for num in step_numbers]
    
    # 提取跳过内容
    content_match = re.search(r'SKIPPED_CONTENT:\s*([^\n]+)', analysis_text)
    if content_match:
        skipping_info['skipped_content'] = content_match.group(1).strip()
    
    # 提取跳过位置
    location_match = re.search(r'SKIPPING_LOCATION:\s*([^\n]+)', analysis_text)
    if location_match:
        location_str = location_match.group(1)
        skipping_info['skipping_location'] = location_str.strip()
        
        # 解析从哪个步骤跳到哪个步骤
        step_matches = re.findall(r'step\s*(\d+)', location_str.lower())
        if len(step_matches) >= 2:
            skipping_info['from_step'] = f"step_{step_matches[0]}"
            skipping_info['to_step'] = f"step_{step_matches[1]}"
    
    return skipping_info

print("✅ Cell 3完成：跳步识别函数定义")

### 为跳步位置构建对比路径

In [ ]:
def generate_comparison_paths_with_llm(skipping_info, step_data, step_by_step_response):
    """使用LLM生成完整的对比推理路径"""
    
    if not skipping_info.get('from_step') or not skipping_info.get('to_step'):
        print("❌ 无法确定跳步位置，使用LLM生成默认对比路径")
        return generate_default_comparison_paths_with_llm()
    
    from_step = skipping_info['from_step']
    to_step = skipping_info['to_step']
    skipped_content = skipping_info.get('skipped_content', 'unknown steps')
    
    print(f"🔍 分析跳步: 从 {from_step} 跳到 {to_step}")
    print(f"跳过的内容: {skipped_content}")
    
    # 获取跳步前的上下文
    context_steps = []
    for step in step_data:
        if step['step_number'] == from_step:
            context_steps.append(step['step_text'])
            break
        context_steps.append(step['step_text'])
    
    context_text = " ".join(context_steps)
    
    # 使用LLM生成多种完整的推理路径
    comparison_paths = {
        'full_reasoning_path_a': generate_llm_reasoning_path("detailed", context_text, skipped_content),
        'full_reasoning_path_b': generate_llm_reasoning_path("alternative", context_text, skipped_content),
        'full_reasoning_path_c': generate_llm_reasoning_path("step_by_step", context_text, skipped_content),
        'pure_shortcut_path': generate_llm_shortcut_path(context_text, skipped_content),
        'skipped_content': skipped_content,
        'jumping_context': context_text
    }
    
    return comparison_paths

def generate_llm_reasoning_path(style, context, skipped_content):
    """使用LLM生成指定风格的完整推理路径"""
    
    if style == "detailed":
        system_msg = "You are a detailed reasoning assistant. Provide complete, logical steps for direction reasoning."
        user_msg = f"""Based on this context: "{context}"

The skipped step was: "{skipped_content}"

Please provide a COMPLETE and DETAILED reasoning path that includes all logical steps. Format as:
Step X: [reasoning]
Step Y: [reasoning]
..."""
    
    elif style == "alternative":
        system_msg = "You are an AI assistant. Provide an alternative complete reasoning path."
        user_msg = f"""Context: "{context}"
Skipped: "{skipped_content}"

Provide an ALTERNATIVE but equally valid complete reasoning path with all steps:"""
    
    else:  # step_by_step
        system_msg = "You are a methodical reasoning assistant."
        user_msg = f"""Given: "{context}"
Missing: "{skipped_content}"

Provide a thorough step-by-step reasoning path with no jumps:"""
    
    prompt = build_llama3_prompt(system_msg, user_msg)
    
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            return_dict_in_generate=True
        )
    
    response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    reasoning_path = parse_llm_reasoning_steps(response)
    
    print(f"  LLM生成{style}路径: {len(reasoning_path)}个步骤")
    return reasoning_path

def generate_llm_shortcut_path(context, skipped_content):
    """使用LLM生成纯跳步路径"""
    
    system_msg = "You are a concise AI assistant that provides direct answers without reasoning."
    user_msg = f"""Based on: "{context}"

Provide ONLY the final conclusion without any intermediate reasoning steps. Be direct and jump to the answer."""
    
    prompt = build_llama3_prompt(system_msg, user_msg)
    
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            return_dict_in_generate=True
        )
    
    response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    shortcut_path = [f"Direct jump: {response.split('assistant')[-1].strip()}"]
    
    print(f"  LLM生成纯跳步路径: {shortcut_path[0][:50]}...")
    return shortcut_path

def parse_llm_reasoning_steps(response_text):
    """解析LLM生成的推理步骤"""
    
    # 提取assistant的响应
    if "assistant" in response_text:
        reasoning_text = response_text.split("assistant")[-1].strip()
    else:
        reasoning_text = response_text
    
    # 使用多种模式匹配步骤
    steps = []
    
    # 模式1: Step X: 格式
    step_pattern1 = r'(?:Step|步骤)\s*(\d+)[:\\.]\\s*([^\\n]+)'
    matches1 = re.findall(step_pattern1, reasoning_text, re.IGNORECASE)
    
    # 模式2: 数字. 格式  
    step_pattern2 = r'(\d+)\\.\\s*([^\\n]+)'
    matches2 = re.findall(step_pattern2, reasoning_text)
    
    # 模式3: 换行分隔的推理步骤
    lines = reasoning_text.split('\n')
    for line in lines:
        line = line.strip()
        if line and (line.startswith('-') or line.startswith('•') or 
                    (len(line) > 10 and not line.startswith('Step'))):
            steps.append(line)
    
    # 优先使用结构化的步骤
    if matches1:
        for num, content in matches1:
            steps.append(f"Step {num}: {content.strip()}")
    elif matches2:
        for num, content in matches2:
            steps.append(f"Step {num}: {content.strip()}")
    
    # 如果还是没有找到步骤，使用整个响应
    if not steps:
        steps = [reasoning_text]
    
    return steps

def generate_default_comparison_paths_with_llm():
    """当无法确定跳步位置时，用LLM生成默认对比路径"""
    
    print("使用LLM生成默认对比路径...")
    
    system_msg = "You are an expert at direction reasoning problems."
    user_msg = f"""Problem: {XP}

Please provide:
1. A detailed complete reasoning path
2. An alternative complete reasoning path  
3. A direct shortcut answer without reasoning

Format as:
DETAILED: [steps]
ALTERNATIVE: [steps]
SHORTCUT: [answer]"""
    
    prompt = build_llama3_prompt(system_msg, user_msg)
    
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            return_dict_in_generate=True
        )
    
    response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    llm_response = response.split("assistant")[-1].strip() if "assistant" in response else response
    
    # 解析LLM的响应
    paths = parse_default_llm_response(llm_response)
    
    return paths

def parse_default_llm_response(response_text):
    """解析LLM生成的默认路径响应"""
    
    paths = {
        'full_reasoning_path_a': [],
        'full_reasoning_path_b': [], 
        'pure_shortcut_path': [],
        'skipped_content': "unknown steps",
        'jumping_context': "default context"
    }
    
    # 尝试解析不同部分
    lines = response_text.split('\n')
    
    detailed_steps = []
    alternative_steps = []
    shortcut_answer = []
    
    current_section = None
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        if 'DETAILED:' in line.upper():
            current_section = 'detailed'
            line = line.split(':', 1)[-1].strip()
        elif 'ALTERNATIVE:' in line.upper():
            current_section = 'alternative'  
            line = line.split(':', 1)[-1].strip()
        elif 'SHORTCUT:' in line.upper():
            current_section = 'shortcut'
            line = line.split(':', 1)[-1].strip()
        
        if current_section == 'detailed' and line:
            detailed_steps.append(line)
        elif current_section == 'alternative' and line:
            alternative_steps.append(line)
        elif current_section == 'shortcut' and line:
            shortcut_answer.append(line)
    
    # 设置路径
    paths['full_reasoning_path_a'] = detailed_steps if detailed_steps else ["Step 1: Default detailed reasoning"]
    paths['full_reasoning_path_b'] = alternative_steps if alternative_steps else ["Step 1: Default alternative reasoning"]
    paths['pure_shortcut_path'] = shortcut_answer if shortcut_answer else ["Direct answer: north"]
    
    return paths

print("✅ Cell 4完成：LLM对比路径生成函数定义")

### 生成路径hidden states并计算相似度

In [ ]:
def generate_paths_hidden_states(comparison_paths, skipping_info):
    """为所有对比路径生成hidden states"""
    
    system_message = "You are a helpful AI assistant that reasons about direction problems."
    path_hidden_states = {}
    
    for path_name, path_steps in comparison_paths.items():
        if path_name == 'skipped_step':
            continue  # 跳过内容描述，不是实际路径
            
        print(f"\n生成路径: {path_name}")
        
        if isinstance(path_steps, list):
            # 为路径中的每个步骤生成hidden states
            step_hidden_states = []
            
            for step_idx, step_text in enumerate(path_steps):
                user_message = f"{XP}\n\nContinue this reasoning:\n{step_text}"
                full_prompt = build_llama3_prompt(system_message, user_message)
                
                inputs = tokenizer(full_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
                
                with torch.no_grad():
                    outputs = model.generate(
                        **inputs,
                        max_new_tokens=5,
                        num_return_sequences=1,
                        do_sample=False,
                        temperature=0.0,
                        output_hidden_states=True,
                        return_dict_in_generate=True
                    )
                
                # 获取hidden state
                last_hidden = outputs.hidden_states[-1][-1][0, -1, :].cpu().numpy()
                step_hidden_states.append(last_hidden)
                
                print(f"  步骤 {step_idx+1}: {step_text[:50]}...")
            
            path_hidden_states[path_name] = step_hidden_states
    
    return path_hidden_states

def calculate_similarity_analysis(skipping_step_hidden, path_hidden_states, step_data, skipping_info):
    """计算跳步位置与各路径的相似度"""
    
    similarity_results = {}
    
    # 找到跳步位置的hidden state
    target_hidden = None
    target_step_name = skipping_info.get('to_step', 'step_3')  # 默认第三个步骤
    
    for step in step_data:
        if step['step_number'] == target_step_name:
            target_hidden = step['hidden_state']
            break
    
    if target_hidden is None and step_data:
        # 使用最后一个步骤作为目标
        target_hidden = step_data[-1]['hidden_state']
    
    if target_hidden is None:
        print("❌ 无法找到目标hidden state")
        return similarity_results
    
    print(f"\n🔬 相似度分析 (目标: {target_step_name})")
    
    # 计算与每个路径的相似度
    for path_name, path_hs_list in path_hidden_states.items():
        path_similarities = []
        
        for path_hs in path_hs_list:
            similarity = cosine_similarity(
                target_hidden.reshape(1, -1),
                path_hs.reshape(1, -1)
            )[0][0]
            path_similarities.append(similarity)
        
        similarity_results[path_name] = {
            'mean_similarity': np.mean(path_similarities),
            'max_similarity': np.max(path_similarities),
            'min_similarity': np.min(path_similarities),
            'all_similarities': path_similarities
        }
        
        print(f"  {path_name}: 平均相似度 = {np.mean(path_similarities):.4f}")
    
    return similarity_results

print("✅ Cell 5完成：相似度分析函数定义")

### 主实验流程

In [ ]:
def main_experiment():
    """主实验流程"""
    
    print("=" * 70)
    print("步骤级别隐式推理分析实验 - 完整流程")
    print("=" * 70)
    
    # 阶段1: 生成分步推理
    print("\n📝 阶段1: 生成分步推理")
    step_by_step_response, step_data = generate_step_by_step_reasoning()
    
    # 阶段2: 识别跳步位置
    print("\n🔍 阶段2: 识别跳步位置")
    skipping_info, judge_analysis = identify_skipped_steps(step_by_step_response)
    
    # 阶段3: 用LLM构建对比路径
    print("\n🛣️  阶段3: 用LLM生成对比路径")
    comparison_paths = generate_comparison_paths_with_llm(skipping_info, step_data, step_by_step_response)
    
    print("LLM生成的对比路径:")
    for path_name, path in comparison_paths.items():
        if isinstance(path, list):
            print(f"  {path_name}:")
            for i, step in enumerate(path):
                print(f"    {i+1}. {step[:60]}..." if len(step) > 60 else f"    {i+1}. {step}")
        else:
            print(f"  {path_name}: {path[:80]}..." if len(path) > 80 else f"  {path_name}: {path}")
    
    # 阶段4: 生成路径hidden states
    print("\n🧠 阶段4: 生成路径hidden states")
    path_hidden_states = generate_paths_hidden_states(comparison_paths, skipping_info)
    
    # 阶段5: 相似度分析
    print("\n📊 阶段5: 相似度分析")
    similarity_results = calculate_similarity_analysis(
        None, path_hidden_states, step_data, skipping_info
    )
    
    # 保存完整结果
    complete_results = {
        'XP': XP,
        'step_by_step_response': step_by_step_response,
        'step_data': step_data,
        'skipping_info': skipping_info,
        'judge_analysis': judge_analysis,
        'comparison_paths': comparison_paths,
        'path_hidden_states': path_hidden_states,
        'similarity_results': similarity_results
    }
    
    save_hidden_states(complete_results, "complete_skipping_analysis.pkl")
    
    # 特别保存跳步位置的hidden state供方法三使用
    jump_step_hidden = None
    target_step_name = skipping_info.get('to_step', 'step_3')
    for step in step_data:
        if step['step_number'] == target_step_name:
            jump_step_hidden = step['hidden_state']
            break
    
    if jump_step_hidden is not None:
        hs_data = {
            'hs_jump': jump_step_hidden,
            'jump_step': target_step_name,
            'XP': XP,
            'skipping_info': skipping_info
        }
        save_hidden_states(hs_data, "method1_jump_hidden_state.pkl")
        print(f"✓ 跳步hidden state已保存，形状: {jump_step_hidden.shape}")
    
    # 输出最终分析结果
    print("\n" + "=" * 70)
    print("🎯 最终分析结果")
    print("=" * 70)
    
    print(f"跳步位置: {skipping_info.get('skipping_location', '未知')}")
    print(f"跳过的内容: {skipping_info.get('skipped_content', '未知')}")
    
    print("\n相似度比较:")
    for path_name, result in similarity_results.items():
        print(f"  {path_name}: {result['mean_similarity']:.4f} (最高: {result['max_similarity']:.4f}, 最低: {result['min_similarity']:.4f})")
    
    # 找出最相似的路径
    if similarity_results:
        best_path = max(similarity_results.items(), key=lambda x: x[1]['mean_similarity'])
        print(f"\n💡 结论: 跳步的内部推理最接近 '{best_path[0]}' 路径")
        print(f"    相似度: {best_path[1]['mean_similarity']:.4f}")
    
    return complete_results

### 执行实验

In [ ]:
print("🚀 开始执行完整实验...")
results = main_experiment()
print("\n🎉 实验完成！")

### 方法二

### 基础设置

In [ ]:
def build_llama3_prompt(system_msg, user_msg):
    """构建符合Llama 3格式的prompt"""
    prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_msg}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_msg}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    return prompt

def save_results(data_dict, filename):
    """保存实验结果"""
    with open(filename, 'wb') as f:
        pickle.dump(data_dict, f)
    print(f"✓ 结果已保存至 {filename}")

print("✅ 方法二基础设置完成")

### logit lens核心函数

In [ ]:
# =====================================================
# Cell 2: Logit Lens核心函数（修复设备问题）
# =====================================================

def get_concept_probability_curve(model, tokenizer, prompt, target_concept):
    """
    获取概念在模型各层的出现概率曲线
    """
    device = model.device
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
    
    # 获取目标概念的token ID
    concept_tokens = tokenizer.encode(target_concept, add_special_tokens=False)
    if len(concept_tokens) == 0:
        raise ValueError(f"无法编码概念: {target_concept}")
    concept_token_id = concept_tokens[0]
    
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
        hidden_states = outputs.hidden_states
        
        lm_head = model.lm_head
        lm_weight = lm_head.weight.to(device)  # 确保权重在正确设备上
        
        probabilities = []
        layers = []
        
        for layer_idx, hidden_state in enumerate(hidden_states):
            # 使用最后一个token的hidden state
            target_hidden = hidden_state[0, -1, :].to(device)  # 确保在正确设备上
            logits = torch.matmul(target_hidden, lm_weight.T)
            probs = torch.softmax(logits, dim=0)
            concept_prob = probs[concept_token_id].item()
            
            probabilities.append(concept_prob)
            layers.append(layer_idx)
    
    return np.array(probabilities), np.array(layers)

### LLM生成验证问题

In [ ]:
def generate_validation_question(model, tokenizer, xp, intermediate_concept, final_answer):
    """
    用LLM生成只能通过中间概念推理得到的验证问题
    """
    system_msg = "You are an expert at creating reasoning validation questions."
    
    user_msg = f"""Base problem: {xp}

We are testing whether the model implicitly reasoned about the intermediate concept: "{intermediate_concept}"

Create a question that:
1. Can ONLY be answered if you know "{intermediate_concept}"
2. Cannot be answered directly from the base problem or final answer "{final_answer}"
3. Requires actual reasoning using the intermediate concept

Provide just the question:"""

    prompt = build_llama3_prompt(system_msg, user_msg)
    
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            return_dict_in_generate=True
        )
    
    response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    question = response.split("assistant")[-1].strip() if "assistant" in response else response
    
    print(f"LLM生成的验证问题: {question}")
    return question

print("✅ 验证问题生成函数完成")

### 构建标准曲线

In [ ]:
def build_standard_curves(model, tokenizer, xp, intermediate_concept, validation_question):
    """
    构建完整推理路径和纯跳步路径的标准曲线
    """
    print("🔧 构建标准曲线...")
    
    # 1. 完整推理路径
    full_reasoning = f"{xp}\n\nStep-by-step reasoning:\n1. Enter from east, walking west → facing west\n2. First right turn → facing {intermediate_concept}\n3. Second right turn → facing east\n4. Left turn → facing north\n\nNow: {validation_question}"
    
    # 2. 纯跳步路径  
    shortcut_reasoning = f"{xp}\n\nAfter all turns, facing north.\n\nNow: {validation_question}"
    
    # 获取概率曲线
    curves = {}
    
    try:
        full_curve, layers = get_concept_probability_curve(model, tokenizer, full_reasoning, intermediate_concept)
        curves['full_reasoning'] = (full_curve, layers)
        print(f"完整路径曲线: 平均概率 = {np.mean(full_curve):.4f}")
    except Exception as e:
        print(f"完整路径错误: {e}")
    
    try:
        shortcut_curve, layers = get_concept_probability_curve(model, tokenizer, shortcut_reasoning, intermediate_concept)
        curves['pure_shortcut'] = (shortcut_curve, layers)
        print(f"纯跳步曲线: 平均概率 = {np.mean(shortcut_curve):.4f}")
    except Exception as e:
        print(f"纯跳步错误: {e}")
    
    return curves

print("✅ 标准曲线构建函数完成")

### 测试曲线提取

In [ ]:
def extract_test_curve(model, tokenizer, xp, jumping_reasoning, validation_question, intermediate_concept):
    """
    为跳步推理提取测试曲线
    """
    print("🔍 提取测试曲线...")
    
    test_prompt = f"{xp}\n\n{jumping_reasoning}\n\nNow: {validation_question}"
    
    try:
        test_curve, layers = get_concept_probability_curve(model, tokenizer, test_prompt, intermediate_concept)
        print(f"测试曲线: 平均概率 = {np.mean(test_curve):.4f}")
        return test_curve, layers
    except Exception as e:
        print(f"测试曲线错误: {e}")
        return None, None

print("✅ 测试曲线提取函数完成")

### 相似度分析

In [ ]:
def analyze_curve_similarity(test_curve, standard_curves):
    """
    分析测试曲线与标准曲线的相似度
    """
    if test_curve is None:
        return {}
    
    similarities = {}
    
    for curve_name, (std_curve, _) in standard_curves.items():
        # 确保长度一致
        min_len = min(len(test_curve), len(std_curve))
        test_trimmed = test_curve[:min_len]
        std_trimmed = std_curve[:min_len]
        
        # 计算余弦相似度
        cosine_sim = cosine_similarity(test_trimmed.reshape(1, -1), std_trimmed.reshape(1, -1))[0][0]
        
        # 计算趋势相似度
        test_diff = np.diff(test_trimmed)
        std_diff = np.diff(std_trimmed)
        if len(test_diff) > 0 and len(std_diff) > 0:
            trend_sim = np.corrcoef(test_diff, std_diff)[0, 1] if not np.isnan(np.corrcoef(test_diff, std_diff)[0, 1]) else 0
        else:
            trend_sim = 0
        
        similarities[curve_name] = {
            'cosine_similarity': cosine_sim,
            'trend_similarity': trend_sim,
            'overall_similarity': (cosine_sim + max(0, trend_sim)) / 2
        }
        
        print(f"与 {curve_name} 的相似度:")
        print(f"  余弦相似度: {cosine_sim:.4f}")
        print(f"  趋势相似度: {trend_sim:.4f}")
        print(f"  总体相似度: {similarities[curve_name]['overall_similarity']:.4f}")
    
    return similarities

print("✅ 相似度分析函数完成")

### 方法二主实验

In [ ]:
def method2_main_experiment(model, tokenizer, xp, intermediate_concept, final_answer, jumping_reasoning):
    """
    方法二主实验流程
    """
    print("=" * 70)
    print("方法二：Logit Lens隐式推理验证")
    print("=" * 70)
    
    # 1. 生成验证问题
    print("\n📝 生成验证问题...")
    validation_question = generate_validation_question(model, tokenizer, xp, intermediate_concept, final_answer)
    
    # 2. 构建标准曲线
    print("\n📈 构建标准曲线...")
    standard_curves = build_standard_curves(model, tokenizer, xp, intermediate_concept, validation_question)
    
    # 3. 提取测试曲线
    print("\n🔍 提取测试曲线...")
    test_curve, test_layers = extract_test_curve(model, tokenizer, xp, jumping_reasoning, validation_question, intermediate_concept)
    
    # 4. 可视化对比
    print("\n📊 可视化对比...")
    if test_curve is not None:
        # 准备绘图数据
        plot_data = {}
        for name, (curve, layers) in standard_curves.items():
            plot_data[name] = (curve, layers)
        plot_data['test_jumping'] = (test_curve, test_layers)
        
        plot_curves(plot_data, f"概念 '{intermediate_concept}' 的概率曲线对比")
    
    # 5. 相似度分析
    print("\n🔬 相似度分析...")
    similarities = analyze_curve_similarity(test_curve, standard_curves)
    
    # 6. 得出结论
    print("\n🎯 结论:")
    if similarities:
        best_match = max(similarities.items(), key=lambda x: x[1]['overall_similarity'])
        best_sim = best_match[1]['overall_similarity']
        
        print(f"跳步推理最接近: {best_match[0]}")
        print(f"相似度: {best_sim:.4f}")
        
        if 'shortcut' in best_match[0]:
            print("💡 模型在跳步时**没有**隐式推理这个概念")
        else:
            print("💡 模型在跳步时**已经**隐式推理了这个概念")
    else:
        print("❌ 无法得出有效结论")
    
    # 保存结果
    results = {
        'xp': xp,
        'intermediate_concept': intermediate_concept,
        'validation_question': validation_question,
        'standard_curves': standard_curves,
        'test_curve': test_curve,
        'similarities': similarities,
        'jumping_reasoning': jumping_reasoning
    }
    
    save_results(results, "method2_results.pkl")
    
    return results

print("✅ 方法二主实验函数完成")

### 构造实验

In [ ]:
# =====================================================
# Cell 8: 执行方法二实验（从文件加载方法一结果）
# =====================================================

print("🚀 开始执行方法二实验...")

def load_method1_results(filename="complete_skipping_analysis.pkl"):
    """从文件加载方法一的结果"""
    try:
        with open(filename, "rb") as f:
            method1_results = pickle.load(f)
        print(f"✓ 成功加载方法一结果 from {filename}")
        return method1_results
    except FileNotFoundError:
        print(f"❌ 找不到方法一结果文件: {filename}")
        return None

def extract_concepts_from_method1(method1_results):
    """从方法一结果中精确提取中间概念和最终答案"""
    
    if method1_results is None:
        return "north", "north"  # 默认值
    
    step_data = method1_results.get('step_data', [])
    skipping_info = method1_results.get('skipping_info', {})
    
    intermediate_concept = None
    final_answer = None
    
    # 从跳步信息中提取跳过的内容
    skipped_content = skipping_info.get('skipped_content', '')
    print(f"跳过的内容: {skipped_content}")
    
    # 分析跳过的步骤来确定中间概念
    if 'first right turn' in skipped_content.lower():
        # 第一个右转：west → north
        intermediate_concept = "north"
    elif 'second right turn' in skipped_content.lower():
        # 第二个右转：north → east  
        intermediate_concept = "east"
    elif 'left turn' in skipped_content.lower():
        # 左转：east → north
        intermediate_concept = "north"
    
    # 从步骤数据中提取最终答案
    for step in step_data:
        step_text = step['step_text'].lower()
        
        # 寻找最终答案步骤
        if any(keyword in step_text for keyword in ['final', 'answer', 'conclusion']):
            if 'north' in step_text:
                final_answer = "north"
            elif 'south' in step_text:
                final_answer = "south" 
            elif 'east' in step_text:
                final_answer = "east"
            elif 'west' in step_text:
                final_answer = "west"
            break
    
    # 如果还是无法确定，使用问题逻辑推断
    if intermediate_concept is None:
        # 根据方向推理问题的标准逻辑
        # 进入方向east，走west → 初始方向west
        # 第一个右转：west → north
        intermediate_concept = "north"
    
    if final_answer is None:
        final_answer = "north"  # 标准答案
    
    return intermediate_concept, final_answer

# 加载方法一结果
method1_results = load_method1_results()

if method1_results is None:
    print("❌ 无法加载方法一结果，使用方法二实验")
    # 使用默认值
    XP = "If you enter a building from the east side, walking west, and then take two rights and a left down corridors, which direction are you facing?"
    intermediate_concept = "north"
    final_answer = "north"
    jumping_reasoning = "After entering from east and walking west, then taking two rights and a left, you end up facing north."
else:
    # 从方法一结果中提取信息
    XP = method1_results.get('XP', "If you enter a building from the east side, walking west, and then take two rights and a left down corridors, which direction are you facing?")
    jumping_reasoning = method1_results.get('step_by_step_response', "After entering from east and walking west, then taking two rights and a left, you end up facing north.")
    intermediate_concept, final_answer = extract_concepts_from_method1(method1_results)

print(f"使用的XP: {XP}")
print(f"提取的中间概念: {intermediate_concept}")
print(f"提取的最终答案: {final_answer}")
print(f"跳步推理文本预览: {jumping_reasoning[:100]}...")

# 执行方法二实验
method2_results = method2_main_experiment(
    model, tokenizer, 
    XP, intermediate_concept, final_answer, jumping_reasoning
)

print("\n🎉 方法二实验完成！")

### 方法三

In [ ]:
# =====================================================
# Cell 1: 方法三基础设置和tuned lens导入
# =====================================================

import torch
import pickle
import numpy as np
from tuned_lens import TunedLens
import re

def build_llama3_prompt(system_msg, user_msg):
    """构建符合Llama 3格式的prompt"""
    prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_msg}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_msg}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    return prompt

def save_results(data_dict, filename):
    """保存实验结果"""
    with open(filename, 'wb') as f:
        pickle.dump(data_dict, f)
    print(f"✓ 数据已保存至 {filename}")

print("✅ 方法三基础设置完成")

# =====================================================
# Cell 2: 修复的Tuned Lens性能分计算器
# =====================================================

class FixedTunedLensScorer:
    """修复的Tuned Lens性能分计算器"""
    
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.device = model.device
        
        # 初始化tuned lens
        print("初始化Tuned Lens...")
        try:
            self.tuned_lens = TunedLens.from_model(model)
            print("✓ Tuned Lens初始化成功")
        except Exception as e:
            print(f"❌ Tuned Lens初始化失败: {e}")
            self.tuned_lens = None
    
    def ensure_tensor_on_device(self, tensor):
        """确保tensor在正确的设备上"""
        if isinstance(tensor, np.ndarray):
            tensor = torch.tensor(tensor)
        
        if tensor.device != self.device:
            tensor = tensor.to(self.device)
        
        return tensor
    
    def calculate_performance_score(self, hidden_state, target_answer):
        """
        使用Tuned Lens计算正确答案token的概率
        """
        try:
            # 确保数据在正确设备上
            hidden_state = self.ensure_tensor_on_device(hidden_state)
            
            # 获取目标答案的token
            target_tokens = self.tokenizer.encode(target_answer, add_special_tokens=False)
            if not target_tokens:
                return 0.0, []
            
            target_token = target_tokens[0]  # 取第一个token
            
            with torch.no_grad():
                # 重塑hidden state为 [1, 1, hidden_dim]
                hidden_state_3d = hidden_state.unsqueeze(0).unsqueeze(0)
                
                if self.tuned_lens is not None:
                    # 修复：正确使用tuned lens，不传递layer_idx参数
                    # tuned lens会返回所有层的logits
                    all_layers_logits = self.tuned_lens(hidden_state_3d)
                    # 取最后一层的logits
                    logits = all_layers_logits[-1][0, -1, :]  # [vocab_size]
                else:
                    # 备用方案：直接使用LM head
                    lm_head = self.model.lm_head
                    logits = lm_head(hidden_state_3d)[0, -1, :]
                
                # 计算概率分布
                probs = torch.softmax(logits, dim=0)
                
                # 获取正确答案token的概率
                correct_token_prob = probs[target_token].item()
                
            return correct_token_prob, [target_token]
            
        except Exception as e:
            print(f"性能分计算错误: {e}")
            return 0.0, []
    
    def get_direct_answer_probability(self, prompt, target_answer):
        """直接对完整prompt计算正确答案token的概率"""
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states=True)
            last_hidden = outputs.hidden_states[-1][0, -1, :]
            
            probability, _ = self.calculate_performance_score(last_hidden, target_answer)
        
        return probability

print("✅ 修复的Tuned Lens性能分计算器完成")

# =====================================================
# Cell 3: 用LLM生成复杂验证问题和直接答案
# =====================================================

def generate_complex_problems_with_answers(model, tokenizer, base_xp, intermediate_concepts):
    """
    用LLM生成复杂验证问题和直接答案（无推理过程）
    """
    system_msg = "You are an AI assistant that provides direct answers without reasoning steps."
    
    user_msg = f"""Base scenario: {base_xp}

Generate TWO complex problems and provide ONLY the direct answers (no reasoning):

1. Problem requiring knowledge of "{intermediate_concepts[0]}":
   - Create a complex question that can ONLY be solved if you know "{intermediate_concepts[0]}"
   - Provide ONLY the final answer

2. Problem requiring knowledge of "{intermediate_concepts[1]}":
   - Create a complex question that can ONLY be solved if you know "{intermediate_concepts[1]}"  
   - Provide ONLY the final answer

Format as:
PROBLEM_A: [question 1]
ANSWER_A: [direct answer 1]
PROBLEM_B: [question 2]  
ANSWER_B: [direct answer 2]

DO NOT include any reasoning steps, explanations, or step-by-step solutions."""

    prompt = build_llama3_prompt(system_msg, user_msg)
    
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            return_dict_in_generate=True
        )
    
    response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    llm_response = response.split("assistant")[-1].strip() if "assistant" in response else response
    
    print("LLM生成的复杂问题和直接答案:")
    print(llm_response)
    
    # 解析问题和答案
    problems = {
        'P1': {'required_concept': intermediate_concepts[0]},
        'P2': {'required_concept': intermediate_concepts[1]}
    }
    
    lines = llm_response.split('\n')
    
    for line in lines:
        line = line.strip()
        if 'PROBLEM_A:' in line:
            problems['P1']['problem'] = line.split('PROBLEM_A:')[-1].strip()
        elif 'ANSWER_A:' in line:
            problems['P1']['answer'] = line.split('ANSWER_A:')[-1].strip()
        elif 'PROBLEM_B:' in line:
            problems['P2']['problem'] = line.split('PROBLEM_B:')[-1].strip()
        elif 'ANSWER_B:' in line:
            problems['P2']['answer'] = line.split('ANSWER_B:')[-1].strip()
    
    # 设置默认值（如果LLM没有生成）
    for key in ['P1', 'P2']:
        if 'problem' not in problems[key]:
            concept = problems[key]['required_concept']
            problems[key]['problem'] = f"If you are facing {concept}, what direction is behind you?"
            problems[key]['answer'] = "south" if concept == "north" else "west"
    
    # 打印最终问题
    for key, data in problems.items():
        print(f"  {key}: {data['problem']}")
        print(f"    直接答案: {data['answer']}")
        print(f"    需要概念: {data['required_concept']}")
    
    return problems

print("✅ 复杂问题和直接答案生成完成")

# =====================================================
# Cell 4: 可行性验证实验
# =====================================================

def feasibility_validation(scorer, base_xp, complex_problems, hs_jump):
    """
    进行可行性验证：测试正确答案token的概率
    """
    print("🔍 进行可行性验证...")
    
    feasibility_results = {}
    
    # 测试1: 直接提供中间概念的基线概率
    concept = complex_problems['P1']['required_concept']
    problem = complex_problems['P1']['problem']
    answer = complex_problems['P1']['answer']
    
    baseline_prompt = f"{base_xp}\n\nWe know: {concept}\n\nProblem: {problem}"
    baseline_prob = scorer.get_direct_answer_probability(baseline_prompt, answer)
    feasibility_results['baseline_probability'] = baseline_prob
    print(f"基线概率 (提供概念{concept}): {baseline_prob:.4f}")
    
    # 测试2: 使用hidden state的概率
    try:
        # 构建只有问题的prompt来获取问题编码的hidden state
        problem_only_prompt = f"{base_xp}\n\nProblem: {problem}"
        inputs = scorer.tokenizer(problem_only_prompt, return_tensors="pt").to(scorer.device)
        
        with torch.no_grad():
            outputs = scorer.model(**inputs, output_hidden_states=True)
            problem_hidden = outputs.hidden_states[-1][0, -1, :]
        
        # 组合hidden states (平均)
        hs_jump_tensor = scorer.ensure_tensor_on_device(hs_jump)
        combined_hidden = (hs_jump_tensor + problem_hidden) / 2
        
        hidden_state_prob, _ = scorer.calculate_performance_score(combined_hidden, answer)
        feasibility_results['hidden_state_probability'] = hidden_state_prob
        print(f"Hidden state概率: {hidden_state_prob:.4f}")
        
    except Exception as e:
        print(f"Hidden state测试错误: {e}")
        feasibility_results['hidden_state_probability'] = 0.0
    
    # 判断可行性
    feasibility_results['is_feasible'] = (
        feasibility_results['baseline_probability'] > 0.1 and 
        feasibility_results['hidden_state_probability'] > 0.05
    )
    
    print(f"实验可行性: {feasibility_results['is_feasible']}")
    
    return feasibility_results

print("✅ 可行性验证完成")

# =====================================================
# Cell 5: 路径推断交叉检验
# =====================================================

def cross_validation_experiment(scorer, base_xp, complex_problems, hs_jump):
    """
    执行路径推断的交叉检验
    """
    print("🔄 执行路径推断交叉检验...")
    
    validation_results = {}
    test_probs = {}
    
    # 检验组: 测试hs_jump在不同问题上的正确答案概率
    for problem_key, problem_data in complex_problems.items():
        problem_text = problem_data['problem']
        target_answer = problem_data['answer']
        required_concept = problem_data['required_concept']
        
        # 构建测试prompt
        test_prompt = f"{base_xp}\n\nProblem: {problem_text}"
        
        # 获取问题编码的hidden state
        inputs = scorer.tokenizer(test_prompt, return_tensors="pt").to(scorer.device)
        with torch.no_grad():
            outputs = scorer.model(**inputs, output_hidden_states=True)
            problem_hidden = outputs.hidden_states[-1][0, -1, :]
        
        # 组合hidden states并计算正确答案概率
        hs_jump_tensor = scorer.ensure_tensor_on_device(hs_jump)
        combined_hidden = (hs_jump_tensor + problem_hidden) / 2
        
        correct_prob, predictions = scorer.calculate_performance_score(combined_hidden, target_answer)
        
        test_probs[problem_key] = {
            'correct_probability': correct_prob,
            'required_concept': required_concept,
            'target_answer': target_answer,
            'predictions': predictions
        }
        
        print(f"问题 {problem_key} (需要{required_concept}) 正确答案概率: {correct_prob:.4f}")
    
    validation_results['test_probs'] = test_probs
    
    # 对照组: 确立概率上限
    print("\n确立概率上限...")
    control_probs = {}
    
    for problem_key, problem_data in complex_problems.items():
        concept = problem_data['required_concept']
        problem_text = problem_data['problem']
        target_answer = problem_data['answer']
        
        # 生成包含明确概念的prompt
        explicit_prompt = f"{base_xp}\n\nWe know: {concept}\n\nProblem: {problem_text}"
        control_prob = scorer.get_direct_answer_probability(explicit_prompt, target_answer)
        control_probs[concept] = control_prob
        print(f"明确提供概念 {concept} 的正确答案概率: {control_prob:.4f}")
    
    validation_results['control_probs'] = control_probs
    
    return validation_results

print("✅ 交叉检验完成")

# =====================================================
# Cell 6: 结果分析和结论
# =====================================================

def analyze_cross_validation_results(validation_results, intermediate_concepts):
    """
    分析交叉检验结果并得出结论
    """
    print("\n📊 分析交叉检验结果...")
    
    test_probs = validation_results['test_probs']
    control_probs = validation_results['control_probs']
    
    # 提取正确答案概率
    prob_E = test_probs['P1']['correct_probability']  # 需要概念A1的问题
    prob_F = test_probs['P2']['correct_probability']  # 需要概念A2的问题
    
    print(f"正确答案概率对比:")
    print(f"  问题P1 (需要{intermediate_concepts[0]}): {prob_E:.4f}")
    print(f"  问题P2 (需要{intermediate_concepts[1]}): {prob_F:.4f}")
    
    # 概率上限参考
    control_A1 = control_probs.get(intermediate_concepts[0], 0)
    control_A2 = control_probs.get(intermediate_concepts[1], 0)
    print(f"概率上限参考:")
    print(f"  明确提供{intermediate_concepts[0]}: {control_A1:.4f}")
    print(f"  明确提供{intermediate_concepts[1]}: {control_A2:.4f}")
    
    # 得出结论
    print("\n🎯 实验结论:")
    
    threshold = 0.05  # 概率差异阈值
    
    if abs(prob_E - prob_F) < threshold:
        print("💡 证据支持'纯跳步'假设")
        print("   - hs_jump中的信息是混沌或无效的")
        print("   - 在两个问题上的正确答案概率无显著差异")
        
    elif prob_E > prob_F + threshold:
        print("💡 强证据表明hs_jump中编码了概念A1的信息")
        print(f"   - 在需要{intermediate_concepts[0]}的问题上正确答案概率更高")
        print(f"   - 概率差异: {prob_E - prob_F:.4f}")
        
    elif prob_F > prob_E + threshold:
        print("💡 强证据表明hs_jump中编码了概念A2的信息") 
        print(f"   - 在需要{intermediate_concepts[1]}的问题上正确答案概率更高")
        print(f"   - 概率差异: {prob_F - prob_E:.4f}")
        
    else:
        print("⚠️ 结果不明确，需要进一步分析")
    
    return {
        'probability_E': prob_E,
        'probability_F': prob_F,
        'probability_difference': prob_E - prob_F,
        'absolute_difference': abs(prob_E - prob_F),
        'control_A1': control_A1,
        'control_A2': control_A2,
        'conclusion': 'pure_shortcut' if abs(prob_E - prob_F) < threshold else 
                     'encodes_A1' if prob_E > prob_F else 'encodes_A2'
    }

print("✅ 结果分析完成")

# =====================================================
# Cell 7: 方法三主实验
# =====================================================

def method3_main_experiment(model, tokenizer):
    """
    方法三主实验流程
    """
    print("=" * 70)
    print("方法三：基于Tuned Lens的性能实证分析")
    print("=" * 70)
    
    try:
        # 加载方法一的结果
        with open("method1_jump_hidden_state.pkl", "rb") as f:
            method1_data = pickle.load(f)
        
        hs_jump = method1_data['hs_jump']
        XP = method1_data['XP']
        skipping_info = method1_data['skipping_info']
        
        print(f"成功加载方法一数据")
        print(f"跳步位置: {skipping_info.get('skipping_location', '未知')}")
        print(f"hs_jump形状: {hs_jump.shape}")
        
    except FileNotFoundError:
        print("❌ 找不到方法一的结果文件，请先运行方法一实验")
        print("请确保方法一已运行并生成了 'method1_jump_hidden_state.pkl' 文件")
        return None
    
    # 设置中间概念
    intermediate_concepts = ["north", "east"]
    
    # 初始化修复的Tuned Lens计算器
    scorer = FixedTunedLensScorer(model, tokenizer)
    
    # 阶段1: 生成复杂验证问题和直接答案
    print("\n📝 生成复杂验证问题和直接答案...")
    complex_problems = generate_complex_problems_with_answers(
        model, tokenizer, XP, intermediate_concepts
    )
    
    # 阶段2: 可行性验证
    print("\n🔍 进行可行性验证...")
    feasibility_results = feasibility_validation(scorer, XP, complex_problems, hs_jump)
    
    if not feasibility_results['is_feasible']:
        print("⚠️ 实验设置可行性较低，但继续执行...")
    
    # 阶段3: 路径推断交叉检验
    print("\n🔄 执行交叉检验...")
    validation_results = cross_validation_experiment(scorer, XP, complex_problems, hs_jump)
    
    # 阶段4: 结果分析
    print("\n📊 分析结果...")
    analysis_results = analyze_cross_validation_results(validation_results, intermediate_concepts)
    
    # 保存完整结果
    final_results = {
        'XP': XP,
        'intermediate_concepts': intermediate_concepts,
        'complex_problems': complex_problems,
        'feasibility_results': feasibility_results,
        'validation_results': validation_results,
        'analysis_results': analysis_results,
        'hs_jump_shape': hs_jump.shape,
        'tuned_lens_used': scorer.tuned_lens is not None
    }
    
    save_results(final_results, "method3_tuned_lens_analysis.pkl")
    
    print(f"\n✅ 方法三实验完成！")
    print(f"✓ 使用了Tuned Lens: {scorer.tuned_lens is not None}")
    print(f"✓ 基于正确答案token概率的分析已完成")
    
    return final_results

print("✅ 方法三主实验完成")

# =====================================================
# Cell 8: 执行方法三实验
# =====================================================

print("🚀 开始执行方法三实验...")
method3_results = method3_main_experiment(model, tokenizer)

if method3_results:
    print("\n🎉 方法三实验成功完成！")
else:
    print("\n❌ 方法三实验失败")

### 基础设置和工具函数

In [ ]:
def build_llama3_prompt(system_msg, user_msg):
    """构建符合Llama 3格式的prompt"""
    prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_msg}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_msg}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
    return prompt

def save_results(data_dict, filename):
    """保存实验结果"""
    with open(filename, 'wb') as f:
        pickle.dump(data_dict, f)
    print(f"✓ 结果已保存至 {filename}")

print("✅ 方法三基础设置完成")

### Tuned Lens性能分计算

In [ ]:
class PerformanceScorer:
    """基于Tuned Lens的性能分计算器"""
    
    def __init__(self, model, tokenizer, tuned_lens=None):
        self.model = model
        self.tokenizer = tokenizer
        self.device = model.device
        
        # 初始化tuned lens
        if tuned_lens is None:
            print("初始化默认Tuned Lens...")
            self.tuned_lens = TunedLens.from_model(model)
        else:
            self.tuned_lens = tuned_lens
    
    def calculate_performance_score(self, hidden_state, target_answer, layer_idx=-1):
        """
        计算给定hidden state在目标答案上的性能分
        
        Args:
            hidden_state: 隐藏状态 [hidden_dim]
            target_answer: 目标答案字符串
            layer_idx: 使用的层索引
            
        Returns:
            performance_score: 性能分 (0-1)
            predictions: 预测的token列表
        """
        # 确保数据在相同设备上
        hidden_state = hidden_state.to(self.device)
        
        # 获取目标答案的token
        target_tokens = self.tokenizer.encode(target_answer, add_special_tokens=False)
        if not target_tokens:
            return 0.0, []
        
        # 使用tuned lens进行层间投射
        with torch.no_grad():
            # 重塑hidden state为 [1, 1, hidden_dim]
            hidden_state_3d = hidden_state.unsqueeze(0).unsqueeze(0)
            
            # 使用tuned lens获取logits
            logits = self.tuned_lens(hidden_state_3d, layer_idx=layer_idx)
            logits = logits[0, -1, :]  # 取最后一个位置的logits [vocab_size]
            
            # 计算概率分布
            probs = torch.softmax(logits, dim=0)
            
            # 对目标token序列计算性能分
            correct_predictions = 0
            predictions = []
            
            for i, target_token in enumerate(target_tokens):
                # 获取预测的token（概率最高的token）
                predicted_token = torch.argmax(probs).item()
                predictions.append(predicted_token)
                
                # 检查预测是否正确
                if predicted_token == target_token:
                    correct_predictions += 1
                
                # 对于多token输出，这里简化处理：只计算第一个token的性能
                # 实际应用中可能需要更复杂的序列生成逻辑
                break  # 暂时只计算第一个token
            
            performance_score = correct_predictions / len(target_tokens)
            
        return performance_score, predictions
    
    def score_reasoning_chain(self, prompt, target_answer):
        """
        直接对完整推理链计算性能分
        """
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states=True)
            last_hidden = outputs.hidden_states[-1][0, -1, :]  # 最后一层最后一个token
            
            performance_score, predictions = self.calculate_performance_score(
                last_hidden, target_answer
            )
        
        return performance_score

print("✅ Tuned Lens性能分计算器完成")

### 用LLM生成复杂验证问题

In [ ]:
def generate_complex_validation_problems(model, tokenizer, base_xp, intermediate_concepts):
    """
    用LLM生成复杂的验证问题对
    """
    system_msg = "You are an expert at creating complex reasoning validation problems."
    
    user_msg = f"""Base scenario: {base_xp}

Intermediate concepts: {', '.join(intermediate_concepts)}

Create TWO complex validation problems with these properties:

1. Problem P1: Should require knowledge of "{intermediate_concepts[0]}" to solve correctly
2. Problem P2: Should require knowledge of "{intermediate_concepts[1]}" to solve correctly  
3. Both problems should be complex and non-trivial
4. Provide the expected answer for each problem
5. The problems should be distinct and test different aspects

Format as:
PROBLEM_P1: [complex problem 1]
ANSWER_P1: [expected answer 1]
PROBLEM_P2: [complex problem 2]  
ANSWER_P2: [expected answer 2]"""

    prompt = build_llama3_prompt(system_msg, user_msg)
    
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            return_dict_in_generate=True
        )
    
    response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    llm_response = response.split("assistant")[-1].strip() if "assistant" in response else response
    
    # 解析生成的复杂问题
    problems = parse_complex_problems(llm_response, intermediate_concepts)
    
    return problems

def parse_complex_problems(response_text, intermediate_concepts):
    """解析LLM生成的复杂问题"""
    
    problems = {
        'P1': {
            'problem': f"Complex problem requiring {intermediate_concepts[0]}",
            'answer': "Expected answer for P1",
            'required_concept': intermediate_concepts[0]
        },
        'P2': {
            'problem': f"Complex problem requiring {intermediate_concepts[1]}", 
            'answer': "Expected answer for P2",
            'required_concept': intermediate_concepts[1]
        }
    }
    
    # 尝试从LLM响应中提取问题
    lines = response_text.split('\n')
    current_problem = None
    
    for line in lines:
        line = line.strip()
        if 'PROBLEM_P1:' in line:
            problems['P1']['problem'] = line.split('PROBLEM_P1:')[-1].strip()
            current_problem = 'P1'
        elif 'ANSWER_P1:' in line:
            problems['P1']['answer'] = line.split('ANSWER_P1:')[-1].strip()
        elif 'PROBLEM_P2:' in line:
            problems['P2']['problem'] = line.split('PROBLEM_P2:')[-1].strip()
            current_problem = 'P2'
        elif 'ANSWER_P2:' in line:
            problems['P2']['answer'] = line.split('ANSWER_P2:')[-1].strip()
    
    print("生成的复杂验证问题:")
    for key, problem_data in problems.items():
        print(f"  {key}: {problem_data['problem'][:80]}...")
        print(f"    答案: {problem_data['answer']}")
        print(f"    需要概念: {problem_data['required_concept']}")
    
    return problems

print("✅ 复杂验证问题生成函数完成")


### 可行性验证实验

In [ ]:
def feasibility_validation(scorer, base_xp, intermediate_concepts, complex_problems, hs_A):
    """
    进行可行性验证：确保实验设置有效
    """
    print("🔍 进行可行性验证...")
    
    feasibility_results = {}
    
    # 测试1: 直接提供中间概念的基线性能
    concept_A1 = intermediate_concepts[0]
    problem_P1 = complex_problems['P1']['problem']
    answer_P1 = complex_problems['P1']['answer']
    
    baseline_prompt = f"{base_xp}\n\nWe know: {concept_A1}\n\nProblem: {problem_P1}"
    baseline_score = scorer.score_reasoning_chain(baseline_prompt, answer_P1)
    feasibility_results['baseline_with_concept'] = baseline_score
    print(f"基线性能 (提供概念{concept_A1}): {baseline_score:.4f}")
    
    # 测试2: 使用hidden state的性能
    try:
        # 构建只有问题的prompt来获取hidden state位置
        problem_only_prompt = f"{base_xp}\n\nProblem: {problem_P1}"
        inputs = scorer.tokenizer(problem_only_prompt, return_tensors="pt").to(scorer.device)
        
        with torch.no_grad():
            outputs = scorer.model(**inputs, output_hidden_states=True)
            problem_hidden = outputs.hidden_states[-1][0, -1, :]  # 问题编码的hidden state
        
        # 组合hidden states (简化处理)
        combined_hidden = (hs_A + problem_hidden) / 2
        
        hidden_state_score, _ = scorer.calculate_performance_score(combined_hidden, answer_P1)
        feasibility_results['hidden_state_performance'] = hidden_state_score
        print(f"Hidden state性能: {hidden_state_score:.4f}")
        
    except Exception as e:
        print(f"Hidden state测试错误: {e}")
        feasibility_results['hidden_state_performance'] = 0.0
    
    # 判断可行性
    feasibility_results['is_feasible'] = (
        feasibility_results['baseline_with_concept'] > 0.7 and 
        feasibility_results['hidden_state_performance'] > 0.5
    )
    
    print(f"实验可行性: {feasibility_results['is_feasible']}")
    
    return feasibility_results

print("✅ 可行性验证函数完成")

### 路径推断交叉检验

In [ ]:
def cross_validation_experiment(scorer, base_xp, complex_problems, hs_A, intermediate_concepts):
    """
    执行路径推断的交叉检验
    """
    print("🔄 执行路径推断交叉检验...")
    
    validation_results = {}
    
    # 检验组: 测试hs_A在不同问题上的性能
    test_scores = {}
    
    for problem_key, problem_data in complex_problems.items():
        problem_text = problem_data['problem']
        target_answer = problem_data['answer']
        required_concept = problem_data['required_concept']
        
        # 构建测试prompt
        test_prompt = f"{base_xp}\n\nProblem: {problem_text}"
        
        # 获取问题编码的hidden state
        inputs = scorer.tokenizer(test_prompt, return_tensors="pt").to(scorer.device)
        with torch.no_grad():
            outputs = scorer.model(**inputs, output_hidden_states=True)
            problem_hidden = outputs.hidden_states[-1][0, -1, :]
        
        # 组合hidden states并计算性能
        combined_hidden = (hs_A.to(scorer.device) + problem_hidden) / 2
        performance_score, predictions = scorer.calculate_performance_score(combined_hidden, target_answer)
        
        test_scores[problem_key] = {
            'performance_score': performance_score,
            'required_concept': required_concept,
            'predictions': predictions
        }
        
        print(f"问题 {problem_key} (需要{required_concept}) 性能: {performance_score:.4f}")
    
    validation_results['test_scores'] = test_scores
    
    # 对照组: 确立性能上限
    print("\n确立性能上限...")
    control_scores = {}
    
    for concept in intermediate_concepts:
        # 生成包含明确概念的prompt
        explicit_prompt = f"{base_xp}\n\nWe know: {concept}\n\nProblem: {complex_problems['P1']['problem']}"
        control_score = scorer.score_reasoning_chain(explicit_prompt, complex_problems['P1']['answer'])
        control_scores[concept] = control_score
        print(f"明确提供概念 {concept} 的性能: {control_score:.4f}")
    
    validation_results['control_scores'] = control_scores
    
    return validation_results

print("✅ 交叉检验函数完成")

### 结果分析和结论

In [ ]:
def analyze_cross_validation_results(validation_results, intermediate_concepts):
    """
    分析交叉检验结果并得出结论
    """
    print("\n📊 分析交叉检验结果...")
    
    test_scores = validation_results['test_scores']
    control_scores = validation_results['control_scores']
    
    # 提取性能分数
    perf_E = test_scores['P1']['performance_score']  # 需要概念A1的问题
    perf_F = test_scores['P2']['performance_score']  # 需要概念A2的问题
    
    print(f"性能分数对比:")
    print(f"  问题P1 (需要{intermediate_concepts[0]}): {perf_E:.4f}")
    print(f"  问题P2 (需要{intermediate_concepts[1]}): {perf_F:.4f}")
    
    # 性能上限参考
    control_A1 = control_scores.get(intermediate_concepts[0], 0)
    control_A2 = control_scores.get(intermediate_concepts[1], 0)
    print(f"性能上限参考:")
    print(f"  明确提供{intermediate_concepts[0]}: {control_A1:.4f}")
    print(f"  明确提供{intermediate_concepts[1]}: {control_A2:.4f}")
    
    # 得出结论
    print("\n🎯 实验结论:")
    
    threshold = 0.1  # 显著性阈值
    
    if abs(perf_E - perf_F) < threshold:
        print("💡 证据支持'纯跳步'假设")
        print("   - hs_A中的信息是混沌或无效的")
        print("   - 在两个问题上的性能无显著差异")
        
    elif perf_E > perf_F + threshold:
        print("💡 强证据表明hs_A中编码了概念A1的信息")
        print(f"   - 在需要{intermediate_concepts[0]}的问题上表现更好")
        print(f"   - 性能差异: {perf_E - perf_F:.4f}")
        
    elif perf_F > perf_E + threshold:
        print("💡 强证据表明hs_A中编码了概念A2的信息") 
        print(f"   - 在需要{intermediate_concepts[1]}的问题上表现更好")
        print(f"   - 性能差异: {perf_F - perf_E:.4f}")
        
    else:
        print("⚠️ 结果不明确，需要进一步分析")
    
    return {
        'performance_difference': perf_E - perf_F,
        'absolute_difference': abs(perf_E - perf_F),
        'conclusion': 'pure_shortcut' if abs(perf_E - perf_F) < threshold else 
                     'encodes_A1' if perf_E > perf_F else 'encodes_A2'
    }

print("✅ 结果分析函数完成")

### 方法三主实验

In [ ]:
def method3_main_experiment(model, tokenizer, base_xp, hs_A, intermediate_concepts):
    """
    方法三主实验流程
    """
    print("=" * 70)
    print("方法三：基于行为表现的路径推断")
    print("=" * 70)
    
    # 初始化性能计算器
    scorer = PerformanceScorer(model, tokenizer)
    
    # 阶段1: 生成复杂验证问题
    print("\n📝 生成复杂验证问题...")
    complex_problems = generate_complex_validation_problems(
        model, tokenizer, base_xp, intermediate_concepts
    )
    
    # 阶段2: 可行性验证
    print("\n🔍 进行可行性验证...")
    feasibility_results = feasibility_validation(
        scorer, base_xp, intermediate_concepts, complex_problems, hs_A
    )
    
    if not feasibility_results['is_feasible']:
        print("❌ 实验设置不可行，终止实验")
        return None
    
    # 阶段3: 路径推断交叉检验
    print("\n🔄 执行交叉检验...")
    validation_results = cross_validation_experiment(
        scorer, base_xp, complex_problems, hs_A, intermediate_concepts
    )
    
    # 阶段4: 结果分析
    print("\n📊 分析结果...")
    analysis_results = analyze_cross_validation_results(validation_results, intermediate_concepts)
    
    # 保存完整结果
    final_results = {
        'base_xp': base_xp,
        'intermediate_concepts': intermediate_concepts,
        'complex_problems': complex_problems,
        'feasibility_results': feasibility_results,
        'validation_results': validation_results,
        'analysis_results': analysis_results,
        'hs_A_shape': hs_A.shape if hasattr(hs_A, 'shape') else str(type(hs_A))
    }
    
    save_results(final_results, "method3_performance_analysis.pkl")
    
    print(f"\n✅ 方法三实验完成！")
    
    return final_results

print("✅ 方法三主实验函数完成")

### 执行方法三实验

In [ ]:
from tuned_lens import TunedLens

In [ ]:
try:
    with open("complete_skipping_analysis.pkl", "rb") as f:
        method1_results = pickle.load(f)
    
    hs_A = method1_results['hs_A']  # 从方法一获取的hidden state
    XP = "If you enter a building from the east side, walking west, and then take two rights and a left down corridors, which direction are you facing?"
    
    # 中间概念
    INTERMEDIATE_CONCEPTS = ["north", "east"]  # 第一个右转和第二个右转后的方向
    
    print("🚀 开始执行方法三实验...")
    method3_results = method3_main_experiment(
        model, tokenizer, XP, hs_A, INTERMEDIATE_CONCEPTS
    )
    
except FileNotFoundError:
    print("❌ 找不到方法一的结果文件，请先运行方法一实验")
except Exception as e:
    print(f"❌ 执行方法三实验时出错: {e}")

print("\n🎉 方法三实验流程完成！")

### 暂时废弃

### 方法三
但是测试结果0.000，需要进一步修改

In [ ]:
import torch
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# --------------------------
# 1. 生成分步推理（让模型显式输出step1, step2...）
# --------------------------

def generate_step_by_step_reasoning():
    """让模型进行分步推理，记录每个步骤的hidden states"""
    
    step_by_step_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful AI assistant that solves direction reasoning problems step by step. 
You MUST output your reasoning in the exact format: Step 1: [reasoning], Step 2: [reasoning], etc.
End with "Final Answer: [direction]"<|eot_id|><|start_header_id|>user<|end_header_id|>

Problem: {XP}

Please solve this step by step and output each step clearly:<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
    
    print("生成分步推理过程...")
    inputs = tokenizer(step_by_step_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7,
            output_hidden_states=True,
            return_dict_in_generate=True
        )
    
    # 获取完整响应
    full_response = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    assistant_response = full_response.split("assistant")[-1].strip() if "assistant" in full_response else full_response
    
    print(f"模型分步推理响应:\n{assistant_response}")
    
    # 提取每个生成步骤的hidden states和对应的文本
    step_data = extract_step_level_hidden_states(outputs, inputs.input_ids.shape[1])
    
    return assistant_response, step_data

def extract_step_level_hidden_states(outputs, prompt_length):
    """提取每个生成步骤的hidden states和对应文本"""
    
    # outputs.hidden_states 的结构: tuple of (layer1_hidden_states, layer2_hidden_states, ...)
    # 每个layer_hidden_states: tuple of (step1, step2, ...) 每个step: [batch, seq_len, hidden_dim]
    
    generated_ids = outputs.sequences[0, prompt_length:]  # 只取生成的部分
    
    step_data = []
    current_step_text = ""
    step_counter = 1
    step_boundaries = []
    
    # 首先识别步骤边界
    full_generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    
    # 查找所有步骤开始的位置
    step_starts = []
    for i in range(1, 10):  # 假设最多10个步骤
        step_marker = f"Step {i}:"
        if step_marker in full_generated_text:
            start_idx = full_generated_text.find(step_marker)
            step_starts.append((i, start_idx))
    
    # 添加Final Answer作为最后一个边界
    if "Final Answer:" in full_generated_text:
        final_idx = full_generated_text.find("Final Answer:")
        step_starts.append(("final", final_idx))
    
    # 按位置排序
    step_starts.sort(key=lambda x: x[1])
    
    # 如果没有检测到步骤结构，整个作为一个步骤
    if not step_starts:
        # 获取最后一个token的hidden state
        last_layer_idx = -1
        last_step_idx = len(outputs.hidden_states[last_layer_idx]) - 1
        last_hidden = outputs.hidden_states[last_layer_idx][last_step_idx][0, -1, :].cpu().numpy()
        
        step_data.append({
            'step_number': 1,
            'step_text': full_generated_text,
            'hidden_state': last_hidden,
            'token_index': len(generated_ids) - 1
        })
        return step_data
    
    # 为每个步骤提取hidden states
    for i, (step_num, start_idx) in enumerate(step_starts):
        # 找到步骤结束的位置
        if i + 1 < len(step_starts):
            end_idx = step_starts[i + 1][1]
            step_text = full_generated_text[start_idx:end_idx].strip()
        else:
            step_text = full_generated_text[start_idx:].strip()
        
        # 找到这个步骤对应的token位置
        step_tokens = tokenizer.encode(step_text, add_special_tokens=False)
        
        # 估算这个步骤在生成序列中的大致位置
        # 这是一个简化方法，实际可能需要更精确的匹配
        approx_token_pos = min(prompt_length + start_idx // 4, len(outputs.sequences[0]) - 1)  # 粗略估算
        
        # 获取这个位置的hidden state（使用最后一个生成步骤）
        last_layer_idx = -1
        step_hidden_states = outputs.hidden_states[last_layer_idx]
        
        # 使用步骤结束位置的hidden state
        step_end_pos = min(approx_token_pos + len(step_tokens), len(step_hidden_states) - 1)
        step_hidden = step_hidden_states[step_end_pos][0, -1, :].cpu().numpy()
        
        step_data.append({
            'step_number': step_num if step_num != "final" else "final",
            'step_text': step_text,
            'hidden_state': step_hidden,
            'token_index': step_end_pos
        })
    
    return step_data

# --------------------------
# 2. 生成参考路径（完整推理 vs 纯跳步）
# --------------------------

def generate_reference_paths():
    """生成完整推理路径和纯跳步路径的hidden states"""
    
    reference_paths = {
        "full_reasoning": [
            "Step 1: Initial direction is west (entering from east walking west). ",
            "Step 2: First right turn: west → north. ",
            "Step 3: Second right turn: north → east. ",
            "Step 4: Left turn: east → north. ",
            "Step 5: Final Answer: north"
        ],
        "pure_shortcut": [
            "Step 1: Final Answer: north"
        ]
    }
    
    system_message = "You are a helpful AI assistant that solves direction reasoning problems step by step."
    
    reference_hidden_states = {}
    
    for path_name, steps in reference_paths.items():
        print(f"\n生成参考路径: {path_name}")
        
        # 为每个步骤生成hidden states
        step_hidden_states = []
        
        for step_idx, step_text in enumerate(steps):
            # 构建到当前步骤的完整上下文
            context_steps = "".join(steps[:step_idx + 1])
            user_message = f"{XP}\n\nPlease continue this reasoning:\n{context_steps}"
            
            full_prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_message}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
            
            inputs = tokenizer(full_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
            
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=5,  # 只需要少量生成来获取hidden state
                    num_return_sequences=1,
                    do_sample=False,
                    temperature=0.0,
                    output_hidden_states=True,
                    return_dict_in_generate=True
                )
            
            # 获取最后一个token的hidden state
            last_layer_idx = -1
            last_step_idx = len(outputs.hidden_states[last_layer_idx]) - 1
            last_hidden = outputs.hidden_states[last_layer_idx][last_step_idx][0, -1, :].cpu().numpy()
            
            step_hidden_states.append({
                'step_number': step_idx + 1,
                'step_text': step_text,
                'hidden_state': last_hidden
            })
            
            print(f"  步骤 {step_idx + 1}: {step_text[:50]}...")
        
        reference_hidden_states[path_name] = step_hidden_states
    
    return reference_hidden_states, reference_paths

# --------------------------
# 3. LLM as Judge 判断是否有跳步
# --------------------------

def detect_skipping_steps(step_by_step_response, reference_paths):
    """使用LLM判断推理过程中是否有跳步"""
    
    judge_prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert judge analyzing reasoning processes. Your task is to detect if there are skipped reasoning steps.
Compare the given reasoning with the expected complete reasoning path.<|eot_id|><|start_header_id|>user<|end_header_id|>

Problem: {XP}

Expected Complete Reasoning:
{chr(10).join(reference_paths['full_reasoning'])}

Actual Reasoning:
{step_by_step_response}

Analysis Questions:
1. Does the actual reasoning skip any logical steps compared to the complete reasoning?
2. If steps are skipped, which specific steps are missing?
3. Rate the reasoning completeness on a scale of 1-5 (1=major skipping, 5=complete reasoning)

Please provide your analysis:<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
    
    inputs = tokenizer(judge_prompt, return_tensors="pt", padding=True, truncation=True).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            num_return_sequences=1,
            do_sample=False,
            temperature=0.0,
            return_dict_in_generate=True
        )
    
    # 正确解码输出
    judge_analysis = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    judge_response = judge_analysis.split("assistant")[-1].strip() if "assistant" in judge_analysis else judge_analysis
    
    print(f"LLM Judge分析:\n{judge_response}")
    
    return judge_response

# --------------------------
# 4. 步骤级别相似度分析
# --------------------------

def analyze_step_similarities(step_data, reference_hidden_states):
    """分析每个步骤与参考路径的相似度"""
    
    analysis_results = {}
    
    for step_info in step_data:
        step_num = step_info['step_number']
        step_hs = step_info['hidden_state']
        step_text = step_info['step_text']
        
        print(f"\n分析步骤 {step_num}: {step_text[:100]}...")
        
        step_similarities = {}
        
        # 与完整推理路径的对应步骤比较
        if 'full_reasoning' in reference_hidden_states:
            full_steps = reference_hidden_states['full_reasoning']
            
            # 根据步骤类型选择比较对象
            if "Final Answer" in step_text or step_num == "final":
                # 最终答案与完整推理的最终步骤比较
                compare_step = full_steps[-1]  # 最后一个步骤
            elif "Step" in str(step_num):
                # 普通推理步骤
                step_idx = int(step_num) - 1 if isinstance(step_num, int) else 0
                compare_step_idx = min(step_idx, len(full_steps) - 1)
                compare_step = full_steps[compare_step_idx]
            else:
                # 默认与第一个步骤比较
                compare_step = full_steps[0]
            
            full_step_hs = compare_step['hidden_state']
            
            similarity = cosine_similarity(
                step_hs.reshape(1, -1), 
                full_step_hs.reshape(1, -1)
            )[0][0]
            
            step_similarities['full_reasoning'] = {
                'similarity': similarity,
                'compared_with_step': compare_step['step_number'],
                'reference_text': compare_step['step_text'][:100] + "..."
            }
            print(f"  与完整推理步骤 {compare_step['step_number']} 相似度: {similarity:.4f}")
        
        # 与纯跳步路径比较
        if 'pure_shortcut' in reference_hidden_states:
            shortcut_steps = reference_hidden_states['pure_shortcut']
            shortcut_step_hs = shortcut_steps[0]['hidden_state']  # 纯跳步只有一个步骤
            
            similarity = cosine_similarity(
                step_hs.reshape(1, -1), 
                shortcut_step_hs.reshape(1, -1)
            )[0][0]
            
            step_similarities['pure_shortcut'] = {
                'similarity': similarity,
                'reference_text': shortcut_steps[0]['step_text'][:100] + "..."
            }
            print(f"  与纯跳步相似度: {similarity:.4f}")
        
        analysis_results[step_num] = {
            'step_text': step_text,
            'similarities': step_similarities,
            'hidden_state': step_hs
        }
    
    return analysis_results

# --------------------------
# 工具函数
# --------------------------

def save_hidden_states(data_dict, filename="hidden_states_data.pkl"):
    """保存数据"""
    with open(filename, 'wb') as f:
        pickle.dump(data_dict, f)
    print(f"✓ 数据已保存至 {filename}")

# --------------------------
# 主执行流程
# --------------------------

def main_experiment():
    """主实验流程"""
    
    print("=" * 60)
    print("步骤级别隐式推理分析实验")
    print("=" * 60)
    
    # 1. 生成分步推理
    step_by_step_response, step_data = generate_step_by_step_reasoning()
    
    # 2. 生成参考路径
    reference_hidden_states, reference_paths = generate_reference_paths()
    
    # 3. LLM判断是否有跳步
    judge_analysis = detect_skipping_steps(step_by_step_response, reference_paths)
    
    # 4. 步骤级别相似度分析
    similarity_analysis = analyze_step_similarities(step_data, reference_hidden_states)
    
    # 5. 保存完整结果
    complete_results = {
        'XP': XP,
        'step_by_step_response': step_by_step_response,
        'step_data': step_data,
        'reference_paths': reference_paths,
        'reference_hidden_states': reference_hidden_states,
        'judge_analysis': judge_analysis,
        'similarity_analysis': similarity_analysis
    }
    
    save_hidden_states(complete_results, "step_level_reasoning_analysis.pkl")
    
    print(f"\n✅ 实验完成！")
    print(f"✅ 结果已保存至 step_level_reasoning_analysis.pkl")
    
    # 打印摘要
    print("\n" + "=" * 60)
    print("实验摘要")
    print("=" * 60)
    print(f"推理步骤数量: {len(step_data)}")
    for step_num, analysis in similarity_analysis.items():
        full_sim = analysis['similarities'].get('full_reasoning', {}).get('similarity', 0)
        shortcut_sim = analysis['similarities'].get('pure_shortcut', {}).get('similarity', 0)
        print(f"步骤 {step_num}: 完整推理相似度={full_sim:.4f}, 纯跳步相似度={shortcut_sim:.4f}")
    
    return complete_results

# 执行实验
if __name__ == "__main__":
    XP = "If you enter a building from the east side, walking west, and then take two rights and a left down corridors, which direction are you facing?"
    
    results = main_experiment()

###  prompt test

In [ ]:
prompt2 = f"""### Task: Generate Standard Reasoning Paths (Non-splittable & Explicit CoT)
### Precondition (Xp): {XP}
### Reasoning Path Requirements:
Generate 3 types of reasoning paths. For paths with intermediate steps (Type 2 and 3), use EXPLICIT CoT (Chain-of-Thought) and ensure each step is a minimal logical closed loop (cannot be split).

#### Type 1: Pure Skip Path (No Intermediate Steps)
- Format: "Initial Condition → Conclusion"
- Requirement: Directly connect the problem context to the final fraction (no CoT).

#### Type 2: Complete Path 1 (A→C→D→Conclusion: 3 steps)
- Format: 
Step 1: Initial Condition → Intermediate C (Calculate Grace's fraction)
Step 2: Intermediate C → Intermediate D (Calculate remaining work after Grace)
Step 3: Intermediate D → Conclusion (Calculate John's fraction after all delegations)
- Requirement: Each step is non-splittable (e.g., "Calculate Grace's fraction" = 1/3 of total work).

#### Type 3: Complete Path 2 (A→E→F→Conclusion: 3 steps)
- Format: 
Step 1: Initial Condition → Intermediate E (Define total work as 1 unit)
Step 2: Intermediate E → Intermediate F (Calculate Andrew's and Beatrice's fractions)
Step 3: Intermediate F → Conclusion (Sum all delegated fractions and subtract from 1)
- Requirement: Each step is non-splittable and logic differs from Type 2 (e.g., starts with total=1).

### Your Output (Clearly label each path type):
"""

# 生成模型输出
inputs = tokenizer(prompt2, return_tensors="pt", padding=True, truncation=True).to(device)
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=500,  # 足够容纳多条路径的显式CoT
        temperature=0.3,  # 降低随机性，确保路径规范性
        do_sample=True
    )

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True).split("### Your Output:")[-1].strip()
print("Generated Result from Prompt 2:")
print(generated_text)

In [ ]:
# --------------------------
# Prompt 2: 生成标准路径（多次）并提取hidden states
# --------------------------
# 1. Prompt配置（明确Type 1-3及显式CoT）
prompt2 = f"""### Task: Generate Standard Reasoning Paths (Non-splittable & Explicit CoT)
### Precondition (Xp): {XP}
### Reasoning Path Requirements:
Generate 3 types of reasoning paths. For paths with intermediate steps (Type 2 and 3), use EXPLICIT CoT and ensure each step is a minimal logical closed loop.

#### Type 1: Pure Skip Path (No Intermediate Steps)
- Format: "Initial Condition → Conclusion"
- Requirement: Directly connect the problem context to the final fraction (no CoT).

#### Type 2: Complete Path 1 (A→C→D→Conclusion: 3 steps)
- Format: 
Step 1: Initial Condition → Intermediate C (Calculate Grace's fraction)
Step 2: Intermediate C → Intermediate D (Calculate remaining work after Grace)
Step 3: Intermediate D → Conclusion (Calculate John's fraction after all delegations)

#### Type 3: Complete Path 2 (A→E→F→Conclusion: 3 steps)
- Format: 
Step 1: Initial Condition → Intermediate E (Define total work as 1 unit)
Step 2: Intermediate E → Intermediate F (Calculate Andrew's and Beatrice's fractions)
Step 3: Intermediate F → Conclusion (Sum all delegated fractions and subtract from 1)

### Your Output (Clearly label each path type):
"""

# 2. 配置生成参数
num_generations = 3  # 每种路径生成3次
target_token_pos = -2  # 目标token位置（结论中最后一个有效token）
extract_layer = -1     # 提取的隐藏层
hidden_states = {      # 存储所有隐藏状态的字典
    "Type 1": [],  # hs_A1, hs_A1', hs_A1''
    "Type 2": [],  # hs_A2, hs_A2', hs_A2''
    "Type 3": []   # hs_A3, hs_A3', hs_A3''
}
generated_paths = {     # 存储所有生成的路径文本
    "Type 1": [],
    "Type 2": [],
    "Type 3": []
}

# 3. 多次生成并提取
for gen_idx in range(num_generations):
    print(f"\n=== Generation {gen_idx + 1}/{num_generations} ===")
    
    # 生成路径
    inputs = tokenizer(prompt2, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=500,
            temperature=0.3,
            do_sample=True,
            output_hidden_states=True,
            return_dict_in_generate=True
        )
    
    # 解码生成结果（使用你的方式）
    full_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
    path_text = full_text.split("### Your Output:")[-1].strip()
    print("Generated Paths:")
    print(path_text)
    
    # 提取每种Type的文本和隐藏状态
    for path_type in ["Type 1", "Type 2", "Type 3"]:
        # 定位该类型在生成文本中的位置（基于标签分割）
        start_idx = path_text.find(f"{path_type}:")
        if start_idx == -1:
            print(f"⚠️ {path_type} not found in generation {gen_idx + 1}")
            continue
        
        # 截取该类型的文本（到下一个Type或结尾）
        end_idx = len(path_text)
        for next_type in ["Type 1", "Type 2", "Type 3"]:
            if next_type != path_type and path_text.find(next_type, start_idx) != -1:
                end_idx = path_text.find(next_type, start_idx)
                break
        type_text = path_text[start_idx:end_idx].strip()
        generated_paths[path_type].append(type_text)
        
        # 提取该类型对应的隐藏状态
        # 重新编码该类型文本以获取准确的hidden states（避免整体序列干扰）
        type_inputs = tokenizer(type_text, return_tensors="pt", padding=True, truncation=True).to(device)
        with torch.no_grad():
            type_outputs = model(** type_inputs, output_hidden_states=True)
        type_hidden = type_outputs.hidden_states[extract_layer]  # [1, seq_len, hidden_dim]
        type_hs = type_hidden[0, target_token_pos, :].cpu().numpy()
        hidden_states[path_type].append(type_hs)
        
        print(f"✅ Extracted {path_type} hidden state (generation {gen_idx + 1})")

# 4. 重命名为要求的格式（hs_A1, hs_A1', hs_A1'' 等）
hs_A1, hs_A1_prime, hs_A1_doubleprime = hidden_states["Type 1"] if len(hidden_states["Type 1"]) == 3 else [None, None, None]
hs_A2, hs_A2_prime, hs_A2_doubleprime = hidden_states["Type 2"] if len(hidden_states["Type 2"]) == 3 else [None, None, None]
hs_A3, hs_A3_prime, hs_A3_doubleprime = hidden_states["Type 3"] if len(hidden_states["Type 3"]) == 3 else [None, None, None]

print("\n=== Hidden States Summary ===")
print(f"Type 1: hs_A1 (shape: {hs_A1.shape if hs_A1 is not None else 'N/A'}), hs_A1' (same), hs_A1'' (same)")
print(f"Type 2: hs_A2 (shape: {hs_A2.shape if hs_A2 is not None else 'N/A'}), hs_A2' (same), hs_A2'' (same)")
print(f"Type 3: hs_A3 (shape: {hs_A3.shape if hs_A3 is not None else 'N/A'}), hs_A3' (same), hs_A3'' (same)")

In [ ]:
all_hidden_states = {
    "hs_A": hs_A,
    "Type 1": [hs_A1, hs_A1_prime, hs_A1_doubleprime],
    "Type 2": [hs_A2, hs_A2_prime, hs_A2_doubleprime],
    "Type 3": [hs_A3, hs_A3_prime, hs_A3_doubleprime]
}

# --------------------------
# 定义相似度计算辅助函数
# --------------------------
def compute_similarity_with_mean(target_hs, reference_hs_list):
    """
    计算目标隐藏状态与参考列表中每个元素的余弦相似度，并返回单个值和平均值
    Args:
        target_hs: 目标隐藏状态（如hs_A）
        reference_hs_list: 参考隐藏状态列表（如[hs_A1, hs_A1', hs_A1'']）
    Returns:
        相似度字典（含单个值和平均值）
    """
    # 过滤无效的隐藏状态（如生成失败的None值）
    valid_refs = [hs for hs in reference_hs_list if hs is not None]
    if not valid_refs:
        return {"similarities": [], "mean_similarity": None}
    
    # 计算每个参考状态的相似度
    similarities = []
    target_norm = target_hs / np.linalg.norm(target_hs) if np.linalg.norm(target_hs) != 0 else target_hs
    for ref_hs in valid_refs:
        ref_norm = ref_hs / np.linalg.norm(ref_hs) if np.linalg.norm(ref_hs) != 0 else ref_hs
        sim = cosine_similarity(target_norm.reshape(1, -1), ref_norm.reshape(1, -1))[0][0]
        similarities.append(round(float(sim), 4))
    
    # 计算平均值
    mean_sim = round(np.mean(similarities), 4) if similarities else None
    return {
        "similarities": similarities,
        "mean_similarity": mean_sim
    }

# --------------------------
# 计算hs_A与每种路径类型的相似度
# --------------------------
# 1. 与Type 1（纯跳步路径）的相似度
type1_results = compute_similarity_with_mean(
    target_hs=hs_A,
    reference_hs_list=all_hidden_states["Type 1"]
)

# 2. 与Type 2（完整路径1）的相似度
type2_results = compute_similarity_with_mean(
    target_hs=hs_A,
    reference_hs_list=all_hidden_states["Type 2"]
)

# 3. 与Type 3（完整路径2）的相似度
type3_results = compute_similarity_with_mean(
    target_hs=hs_A,
    reference_hs_list=all_hidden_states["Type 3"]
)

# --------------------------
# 输出详细结果
# --------------------------
print("="*80)
print("Cosine Similarity Results (hs_A vs Each Path Type)")
print("="*80)

# Type 1 结果
print(f"Type 1 (Pure Skip Path):")
print(f"  Similarities with hs_A1, hs_A1', hs_A1'': {type1_results['similarities']}")
print(f"  Mean Similarity: {type1_results['mean_similarity']}\n")

# Type 2 结果
print(f"Type 2 (Complete Path 1):")
print(f"  Similarities with hs_A2, hs_A2', hs_A2'': {type2_results['similarities']}")
print(f"  Mean Similarity: {type2_results['mean_similarity']}\n")

# Type 3 结果
print(f"Type 3 (Complete Path 2):")
print(f"  Similarities with hs_A3, hs_A3', hs_A3'': {type3_results['similarities']}")
print(f"  Mean Similarity: {type3_results['mean_similarity']}")
print("="*80)

In [ ]:
import pickle

# 保存所有隐藏状态（二进制格式，保留原始数组）
with open("all_hidden_states.pkl", "wb") as f:
    pickle.dump(all_hidden_states, f)

# 保存相似度结果（文本格式，便于查看）
similarity_results = {
    "Type 1": type1_results,
    "Type 2": type2_results,
    "Type 3": type3_results
}
with open("similarity_results.json", "w") as f:
    json.dump(similarity_results, f, indent=4)

print("\n✅ All hidden states saved to 'all_hidden_states.pkl'")
print("✅ Similarity results saved to 'similarity_results.json'")

### hidden states能力简化测试

In [ ]:
# --------------------------
# 加载实验数据（复用Prompt2的hs_A3和A3）
# --------------------------
# 从方法一保存的隐藏状态文件中加载hs_A3（Type3的隐藏状态）
with open("all_hidden_states.pkl", "rb") as f:
    all_hs = pickle.load(f)
hs_A3 = all_hs["Type 3"][0]  # 取Type3第一次生成的hidden state（可替换为其他次）
print(f"✅ Loaded hs_A3 (shape: {hs_A3.shape})")  # shape: [hidden_dim,] 不是层数！

# 定义已知的中间条件A3和测试问题Q（补充完整上下文，避免模型歧义）
A3 = "the sum of delegated fractions is 13/12"  # 你的中间条件A3
# 补充原题背景，让问题更明确（模型需要完整上下文才能推理）
Q = "John has some work he needs to be done by the end of the week. He delegates fractions to others and completes the rest. If the sum of delegated fractions is 13/12, what fraction of the entire work does John do? (Note: Total work is 1, so John's fraction = 1 - sum of delegated fractions)"
correct_conclusion = "John does 1/12 of the work"  # 预期正确结论

print("\n📋 Test Configuration:")
print(f"A3 (Intermediate Condition): {A3}")
print(f"Q (Test Question): {Q}")
print(f"Expected Correct Conclusion: {correct_conclusion}")

# --------------------------
# 构建两种测试输入（优化提示词，引导精准输出）
# --------------------------
# 输入1：A3 + Q（直接提供中间条件，作为基准）
input_a3_q = f"Given the information: {A3}. Based on this, answer the question: {Q} Output only the final conclusion in the format 'John does X of the work' (no extra text)."
# 输入2：hs_A3嵌入 + Q（使用更自然的表述，传递隐藏状态信息）
# 注：hs_A3是[hidden_dim,]的向量，取前15个数值是为了文本嵌入示意，非层数
hs_embedding_text = f"[Task-related semantic embedding: {np.round(hs_A3[:15], 4)}...]"  # 保留4位小数，更简洁
input_hs_q = f"Use the following task-related semantic embedding: {hs_embedding_text}. Based on this embedding, answer the question: {Q} Output only the final conclusion in the format 'John does X of the work' (no extra text)."

print(f"\n🔍 Test Inputs:")
print(f"Input 1 (A3 + Q): {input_a3_q}")
print(f"Input 2 (hs_A3 + Q): {input_hs_q}")

# --------------------------
# 推理函数定义（优化生成参数，避免重复输出）
# --------------------------
def get_model_conclusion(input_text):
    """生成模型对输入的结论输出"""
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=300
    ).to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=0.1,  # 进一步降低随机性
            do_sample=False,  # 禁用采样，使用确定性生成
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.2  # 抑制重复输出
        )
    
    # 提取模型生成的结论（排除输入文本部分）
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    conclusion = full_output.replace(input_text, "").strip()
    return conclusion

# --------------------------
# 执行测试并评估
# --------------------------
print("\n🚀 Generating model conclusions...")
conclusion_a3_q = get_model_conclusion(input_a3_q)
conclusion_hs_q = get_model_conclusion(input_hs_q)

# 结论正确性评估（支持模糊匹配，更灵活）
def is_correct(conclusion, correct_answer):
    correct_keywords = correct_answer.lower().split()
    return all(keyword in conclusion.lower() for keyword in correct_keywords)

correct_a3 = is_correct(conclusion_a3_q, correct_conclusion)
correct_hs = is_correct(conclusion_hs_q, correct_conclusion)

# --------------------------
# 输出测试报告
# --------------------------
print("\n" + "="*80)
print("📊 Hidden States Understanding Test Report")
print("="*80)
print(f"Input 1 (A3 + Q) → Conclusion: {conclusion_a3_q}")
print(f"Input 1 Correct? → {'✅ YES' if correct_a3 else '❌ NO'}")
print(f"\nInput 2 (hs_A3 + Q) → Conclusion: {conclusion_hs_q}")
print(f"Input 2 Correct? → {'✅ YES' if correct_hs else '❌ NO'}")
print("="*80)